In [6]:
import math
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mediapipe as mp
from PIL import Image
import os
from tqdm import tqdm

In [13]:
def euclidean_distance(a, b):
    x1 = a[0]
    y1 = a[1]
    x2 = b[0]
    y2 = b[1]
    
    return math.sqrt((x2 - x1) * (x2 - x1) + (y2 - y1)* (y2 - y1))

In [14]:
def image_preprocessing(path):
    image = cv2.imread(path)
    
    if(type(image) == type(None)):
        #print("Cannot preprocess. image is None.")
        return
    
    # resize (48, 48) -> (224, 224)
    resized_image = resize(image)
    # normalize
    normalized_image = normalize(resized_image)
    # face rotation with extracting roi 
    roi_image = rotation_roi(normalized_image)
    # resize roi.shape -> -> (224, 224)
    resized_roi = resize(roi_image) 
    # apply face mesh
    result = reduced_face_mesh(resized_roi)
    
    return result

In [22]:
def resize(image):
    if(type(image) == type(None)):
        #print("Cannot preprocess. image is None.")
        return
    
    #print("In resize(), type(image)=", type(image))
    image = image.astype(np.uint8)
    resized_image = cv2.resize(image, dsize=(224, 224), interpolation=cv2.INTER_LINEAR)
    return resized_image

In [16]:
def normalize(image):
    if(type(image) == type(None)):
        #print("Cannot normalize. image is None.")
        return
    
    result = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)
    # print(type(result))
    return result

In [17]:
def rotation_roi(image):
    if(type(image) == type(None)):
        #print("Cannot rotation. image is None.")
        return None, None
    
    # the position number of silhoueets
    silhouette = [
        10,  338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288,
        397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136,
        172, 58,  132, 93,  234, 127, 162, 21,  54,  103, 67,  109]
    
    # for ROI
    x, y = 0, 0
    minX, minY = 1000, 1000 
    maxX, maxY = 0, 0
    roi_t = 10
    
    # for face mesh style
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_face_mesh = mp.solutions.face_mesh
    
    # For static images:
    drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
    
    with mp_face_mesh.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5) as face_mesh:
        
        width = image.shape[1]
        height = image.shape[0]
        new_image = image.copy()
        
        # Convert the BGR image to RGB before processing.
        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
        # Print and draw face mesh landmarks on the image.
        if not results.multi_face_landmarks:
              return None
            
        annotated_image = image.copy()

        for face_landmarks in results.multi_face_landmarks:
            ### rotation
            irises = list(mp_face_mesh.FACEMESH_IRISES)
                
            # left eye 눈동자 상하 위치 인덱스: 0, 1 
            l_pt1_x = int(face_landmarks.landmark[irises[0][0]].x * width)
            l_pt1_y = int(face_landmarks.landmark[irises[0][0]].y * height)
            l_pt2_x = int(face_landmarks.landmark[irises[1][0]].x * width)
            l_pt2_y = int(face_landmarks.landmark[irises[1][0]].y * height)
            # right eye 눈동자 상하 위치 인덱스: 3, 6
            r_pt1_x = int(face_landmarks.landmark[irises[7][0]].x * width)
            r_pt1_y = int(face_landmarks.landmark[irises[7][0]].y * height)
            r_pt2_x = int(face_landmarks.landmark[irises[3][0]].x * width)
            r_pt2_y = int(face_landmarks.landmark[irises[3][0]].y * height)

            # left eye
            if l_pt1_x > l_pt2_x:
                left_eye_x = l_pt2_x + (l_pt1_x - l_pt2_x) // 2
            else:
                left_eye_x = l_pt1_x + (l_pt2_x - l_pt1_x) // 2

            left_eye_y = l_pt1_y + (l_pt2_y - l_pt1_y) // 2

            # right eye
            if r_pt1_x > r_pt2_x:
                right_eye_x = r_pt2_x + (r_pt1_x - r_pt2_x) // 2
            else:
                right_eye_x = r_pt1_x + (r_pt2_x - r_pt1_x) // 2

            right_eye_y = r_pt1_y + (r_pt2_y - r_pt1_y) // 2

            left_eye_center = (left_eye_x, left_eye_y)
            right_eye_center = (right_eye_x, right_eye_y)
            
            if left_eye_y < right_eye_y :
                point_3rd = (right_eye_x, left_eye_y)
                direction = 1 # 반시계방향
                # print("반시계방향 회전")
            else:
                point_3rd = (left_eye_x, right_eye_y)
                direction = -1 # 시계방향
                # print("시계방향 회전")

            a = euclidean_distance(left_eye_center, point_3rd)
            b = euclidean_distance(right_eye_center, left_eye_center)
            c = euclidean_distance(right_eye_center, point_3rd)

            cos_a = (b * b + c * c - a * a) / (2 * b * c)
            # print("cos(a) = ", cos_a)

            angle = np.arccos(cos_a)
            # print("angle : " , angle, " in radian")

            angle = (angle * 180) / math.pi
            # print("angle : ", angle, " in degree")

            if direction == 1:
                angle = 90 - angle

        rotated_image = Image.fromarray(annotated_image)
        rotated_image = np.array(rotated_image.rotate(direction * (-1) * angle))
    
    roi_image = roi(face_landmarks, rotated_image)
    
    return roi_image

In [18]:
def roi(face_landmarks, image):
    if(type(image) == type(None)):
        #print("Cannot extract roi. ")
        return _, None
    
     # the position number of silhoueets
    silhouette = [
        10,  338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288,
        397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136,
        172, 58,  132, 93,  234, 127, 162, 21,  54,  103, 67,  109]
    
    # for ROI
    x, y = 0, 0
    minX, minY = 1000, 1000 
    maxX, maxY = 0, 0
    roi_t = 10
    
    height = image.shape[0]
    width = image.shape[1]
    
    for i in silhouette:
        # find silhouette
        x = int(face_landmarks.landmark[i].x * image.shape[1])
        y = int(face_landmarks.landmark[i].y * image.shape[0])
        
        # find ROI coordinate
        if x < minX:
            minX = x
        if y < minY:
            minY = y
        if x > maxX:
            maxX = x
        if y > maxY:
            maxY = y

    # adjust the points (x1, y1), (x2, y2)
    x1 = minX - roi_t
    y1 = minY - roi_t
    x2 = maxX + roi_t + 1
    y2 = maxY + roi_t + 1

    if(x1 < 0):
        x1 = 0
    if(y1 < 0):
        y1 = 0
    if(x2 > width or x2 < 0):
        x2 = width
    if(y2 > height or y2 < 0):
        y2 = height

    # extract roi
    #print(x1, x2, y1, y2)
    roi_image = image[y1:y2, x1:x2]

    return roi_image

In [28]:
def reduced_face_mesh(image):
    if(type(image) == type(None)):
        #print("Cannot face mesh. image is None.")
        return
    
    # for face mesh style
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_face_mesh = mp.solutions.face_mesh
    
    # For static images:
    drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
    
    height = image.shape[0]
    width = image.shape[1]
  
    landmarks = list(mp_face_mesh.FACEMESH_CONTOURS)
    
    with mp_face_mesh.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5) as face_mesh:
        
        # Convert the BGR image to RGB before processing.
        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Print and draw face mesh landmarks on the image.
        if not results.multi_face_landmarks:
            #print("Cannot find landmark on the image.")
            return
            
        annotated_image = image.copy()
        
        for face_landmarks in results.multi_face_landmarks:
            for i in range(len(landmarks)):
                x = int(face_landmarks.landmark[landmarks[i][0]].x * width)
                y = int(face_landmarks.landmark[landmarks[i][0]].y * height)
                
               # print(i)
                cv2.line(annotated_image, (x, y), (x, y), (0, 0, 255), 2)
            
    return annotated_image

In [9]:
path = '../../data/FERPlus/images/FER2013Train/fer0000000.png'
image = cv2.imread(path)
image = resize(image)
image = normalize(image)
image = rotation_roi(image)
image = resize(image)
image = reduced_face_mesh(image)

if(type(image) != type(None)):
    cv2.imwrite("test.png", image)
    print(image.shape)

In resize(), type(image)= <class 'numpy.ndarray'>
In resize(), type(image)= <class 'numpy.ndarray'>
(224, 224, 3)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [20]:
plt.imshow(image)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/sirlab/anaconda3/envs/pytorch/lib/python3.9/site-packages/IPython/core/formatters.py", line 222, in catch_format_error
    r = method(self, *args, **kwargs)
  File "/home/sirlab/anaconda3/envs/pytorch/lib/python3.9/site-packages/IPython/core/formatters.py", line 339, in __call__
    return printer(obj)
  File "/home/sirlab/anaconda3/envs/pytorch/lib/python3.9/site-packages/IPython/core/pylabtools.py", line 151, in print_figure
    fig.canvas.print_figure(bytes_io, **kw)
  File "/home/sirlab/anaconda3/envs/pytorch/lib/python3.9/site-packages/matplotlib/backend_bases.py", line 2319, in print_figure
    result = print_method(
  File "/home/sirlab/anaconda3/envs/pytorch/lib/python3.9/site-packages/matplotlib/backend_bases.py", line 1648, in wrapper
    return func(*args, **kwargs)
  File "/home/sirlab/anaconda3/envs/pytorch/lib/python3.9/site-packages/matplotlib/_api/deprecation.py", line 415, in wrapper
    return func(*inner_args, **inner_

<Figure size 640x480 with 1 Axes>

# Make dataset with landmark

In [35]:
os.chdir("..")

In [29]:
os.chdir("../../../../Capstone_FER/preprocess")

In [30]:
os.getcwd()

'/home/sirlab/FER/Capstone_FER/preprocess'

In [31]:
os.chdir("../../data/FERPlus/images")
#classes = ["neutral", "happy", "fear", "disgust", "angry", "sad", "surprise"]
no_landmark_dict = {}

for mode in ["FER2013Train", "FER2013Test", "FER2013Valid"]:
    print("###", mode, "###")
    no_landmark = {}
    
    os.chdir("./" + mode)
    
#     for label in classes:
#         print("##", label, "##")
#         os.chdir("./"+label)
    files = sorted(os.listdir(os.getcwd()))
    print("Total count of images in {}: {}".format(mode, len(files)))

    file_list = []
    none_file_count = 0

    for file in tqdm(files):
        result = image_preprocessing(file)
        
        if file == 'label.csv':
            continue
            
        if(type(result) == type(None)):
            file_list.append(file)
            none_file_count += 1
            
        else:
            cv2.imwrite("../../../refinedFERPlus/"+mode+"/"+file, result)
            
    print("Total count of non images(cannot preprossed):", none_file_count)
    print("file list which cannot preprocess:")
    print(file_list)
    no_landmark[mode] = file_list

    os.chdir("..")

os.chdir("..")

### FER2013Train ###
Total count of images in FER2013Train: 28559


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28559/28559 [40:20<00:00, 11.80it/s]


Total count of non images(cannot preprossed): 3561
file list which cannot preprocess:
['fer0000003.png', 'fer0000018.png', 'fer0000029.png', 'fer0000038.png', 'fer0000043.png', 'fer0000049.png', 'fer0000067.png', 'fer0000073.png', 'fer0000082.png', 'fer0000114.png', 'fer0000122.png', 'fer0000124.png', 'fer0000130.png', 'fer0000139.png', 'fer0000153.png', 'fer0000157.png', 'fer0000161.png', 'fer0000188.png', 'fer0000199.png', 'fer0000201.png', 'fer0000208.png', 'fer0000233.png', 'fer0000237.png', 'fer0000247.png', 'fer0000253.png', 'fer0000255.png', 'fer0000263.png', 'fer0000271.png', 'fer0000285.png', 'fer0000288.png', 'fer0000295.png', 'fer0000298.png', 'fer0000300.png', 'fer0000307.png', 'fer0000332.png', 'fer0000335.png', 'fer0000348.png', 'fer0000361.png', 'fer0000366.png', 'fer0000367.png', 'fer0000393.png', 'fer0000400.png', 'fer0000403.png', 'fer0000417.png', 'fer0000429.png', 'fer0000436.png', 'fer0000446.png', 'fer0000469.png', 'fer0000470.png', 'fer0000472.png', 'fer0000478.p

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3574/3574 [04:58<00:00, 11.96it/s]


Total count of non images(cannot preprossed): 456
file list which cannot preprocess:
['fer0032220.png', 'fer0032233.png', 'fer0032234.png', 'fer0032236.png', 'fer0032238.png', 'fer0032239.png', 'fer0032253.png', 'fer0032256.png', 'fer0032262.png', 'fer0032271.png', 'fer0032279.png', 'fer0032280.png', 'fer0032289.png', 'fer0032292.png', 'fer0032297.png', 'fer0032312.png', 'fer0032315.png', 'fer0032331.png', 'fer0032332.png', 'fer0032345.png', 'fer0032352.png', 'fer0032359.png', 'fer0032383.png', 'fer0032399.png', 'fer0032403.png', 'fer0032411.png', 'fer0032414.png', 'fer0032428.png', 'fer0032430.png', 'fer0032433.png', 'fer0032440.png', 'fer0032447.png', 'fer0032455.png', 'fer0032456.png', 'fer0032457.png', 'fer0032464.png', 'fer0032468.png', 'fer0032470.png', 'fer0032472.png', 'fer0032473.png', 'fer0032484.png', 'fer0032485.png', 'fer0032493.png', 'fer0032511.png', 'fer0032521.png', 'fer0032527.png', 'fer0032532.png', 'fer0032546.png', 'fer0032548.png', 'fer0032555.png', 'fer0032559.pn

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3580/3580 [05:10<00:00, 11.53it/s]

Total count of non images(cannot preprossed): 476
file list which cannot preprocess:
['fer0028640.png', 'fer0028648.png', 'fer0028653.png', 'fer0028664.png', 'fer0028683.png', 'fer0028694.png', 'fer0028702.png', 'fer0028704.png', 'fer0028706.png', 'fer0028707.png', 'fer0028709.png', 'fer0028718.png', 'fer0028719.png', 'fer0028723.png', 'fer0028732.png', 'fer0028733.png', 'fer0028746.png', 'fer0028749.png', 'fer0028751.png', 'fer0028768.png', 'fer0028781.png', 'fer0028814.png', 'fer0028815.png', 'fer0028816.png', 'fer0028828.png', 'fer0028829.png', 'fer0028845.png', 'fer0028849.png', 'fer0028859.png', 'fer0028862.png', 'fer0028866.png', 'fer0028868.png', 'fer0028873.png', 'fer0028874.png', 'fer0028876.png', 'fer0028895.png', 'fer0028900.png', 'fer0028903.png', 'fer0028908.png', 'fer0028922.png', 'fer0028924.png', 'fer0028934.png', 'fer0028937.png', 'fer0028938.png', 'fer0029007.png', 'fer0029010.png', 'fer0029012.png', 'fer0029021.png', 'fer0029027.png', 'fer0029044.png', 'fer0029060.pn

In [50]:
training_miss = ['fer0000003.png', 'fer0000018.png', 'fer0000029.png', 'fer0000038.png', 'fer0000043.png', 'fer0000049.png', 'fer0000067.png', 'fer0000073.png', 'fer0000082.png', 'fer0000114.png', 'fer0000122.png', 'fer0000124.png', 'fer0000130.png', 'fer0000139.png', 'fer0000153.png', 'fer0000157.png', 'fer0000161.png', 'fer0000188.png', 'fer0000199.png', 'fer0000201.png', 'fer0000208.png', 'fer0000233.png', 'fer0000237.png', 'fer0000247.png', 'fer0000253.png', 'fer0000255.png', 'fer0000263.png', 'fer0000271.png', 'fer0000285.png', 'fer0000288.png', 'fer0000295.png', 'fer0000298.png', 'fer0000300.png', 'fer0000307.png', 'fer0000332.png', 'fer0000335.png', 'fer0000348.png', 'fer0000361.png', 'fer0000366.png', 'fer0000367.png', 'fer0000393.png', 'fer0000400.png', 'fer0000403.png', 'fer0000417.png', 'fer0000429.png', 'fer0000436.png', 'fer0000446.png', 'fer0000469.png', 'fer0000470.png', 'fer0000472.png', 'fer0000478.png', 'fer0000482.png', 'fer0000483.png', 'fer0000485.png', 'fer0000486.png', 'fer0000491.png', 'fer0000492.png', 'fer0000495.png', 'fer0000497.png', 'fer0000511.png', 'fer0000512.png', 'fer0000516.png', 'fer0000527.png', 'fer0000531.png', 'fer0000532.png', 'fer0000538.png', 'fer0000546.png', 'fer0000555.png', 'fer0000558.png', 'fer0000562.png', 'fer0000564.png', 'fer0000573.png', 'fer0000575.png', 'fer0000578.png', 'fer0000604.png', 'fer0000619.png', 'fer0000652.png', 'fer0000653.png', 'fer0000655.png', 'fer0000658.png', 'fer0000659.png', 'fer0000695.png', 'fer0000700.png', 'fer0000719.png', 'fer0000724.png', 'fer0000736.png', 'fer0000741.png', 'fer0000743.png', 'fer0000754.png', 'fer0000756.png', 'fer0000760.png', 'fer0000785.png', 'fer0000797.png', 'fer0000806.png', 'fer0000819.png', 'fer0000826.png', 'fer0000834.png', 'fer0000841.png', 'fer0000842.png', 'fer0000849.png', 'fer0000851.png', 'fer0000862.png', 'fer0000867.png', 'fer0000878.png', 'fer0000883.png', 'fer0000886.png', 'fer0000891.png', 'fer0000909.png', 'fer0000938.png', 'fer0000939.png', 'fer0000947.png', 'fer0000948.png', 'fer0000952.png', 'fer0000955.png', 'fer0000956.png', 'fer0000960.png', 'fer0000961.png', 'fer0000963.png', 'fer0000970.png', 'fer0000975.png', 'fer0000978.png', 'fer0000981.png', 'fer0000983.png', 'fer0001006.png', 'fer0001009.png', 'fer0001011.png', 'fer0001014.png', 'fer0001033.png', 'fer0001034.png', 'fer0001054.png', 'fer0001061.png', 'fer0001067.png', 'fer0001094.png', 'fer0001114.png', 'fer0001115.png', 'fer0001120.png', 'fer0001125.png', 'fer0001133.png', 'fer0001146.png', 'fer0001148.png', 'fer0001156.png', 'fer0001158.png', 'fer0001164.png', 'fer0001166.png', 'fer0001190.png', 'fer0001206.png', 'fer0001214.png', 'fer0001223.png', 'fer0001272.png', 'fer0001274.png', 'fer0001277.png', 'fer0001293.png', 'fer0001299.png', 'fer0001308.png', 'fer0001317.png', 'fer0001326.png', 'fer0001341.png', 'fer0001351.png', 'fer0001356.png', 'fer0001362.png', 'fer0001363.png', 'fer0001370.png', 'fer0001372.png', 'fer0001378.png', 'fer0001393.png', 'fer0001394.png', 'fer0001407.png', 'fer0001410.png', 'fer0001418.png', 'fer0001421.png', 'fer0001429.png', 'fer0001438.png', 'fer0001450.png', 'fer0001454.png', 'fer0001470.png', 'fer0001488.png', 'fer0001489.png', 'fer0001493.png', 'fer0001501.png', 'fer0001509.png', 'fer0001520.png', 'fer0001531.png', 'fer0001535.png', 'fer0001542.png', 'fer0001547.png', 'fer0001554.png', 'fer0001557.png', 'fer0001569.png', 'fer0001584.png', 'fer0001585.png', 'fer0001586.png', 'fer0001588.png', 'fer0001590.png', 'fer0001591.png', 'fer0001593.png', 'fer0001596.png', 'fer0001599.png', 'fer0001617.png', 'fer0001623.png', 'fer0001630.png', 'fer0001633.png', 'fer0001636.png', 'fer0001637.png', 'fer0001640.png', 'fer0001648.png', 'fer0001655.png', 'fer0001658.png', 'fer0001669.png', 'fer0001675.png', 'fer0001686.png', 'fer0001715.png', 'fer0001722.png', 'fer0001727.png', 'fer0001728.png', 'fer0001734.png', 'fer0001738.png', 'fer0001740.png', 'fer0001741.png', 'fer0001748.png', 'fer0001755.png', 'fer0001759.png', 'fer0001765.png', 'fer0001772.png', 'fer0001774.png', 'fer0001778.png', 'fer0001787.png', 'fer0001838.png', 'fer0001845.png', 'fer0001847.png', 'fer0001864.png', 'fer0001899.png', 'fer0001904.png', 'fer0001922.png', 'fer0001923.png', 'fer0001928.png', 'fer0001929.png', 'fer0001933.png', 'fer0001941.png', 'fer0001944.png', 'fer0001947.png', 'fer0001949.png', 'fer0001957.png', 'fer0001964.png', 'fer0001971.png', 'fer0002007.png', 'fer0002008.png', 'fer0002016.png', 'fer0002023.png', 'fer0002027.png', 'fer0002037.png', 'fer0002053.png', 'fer0002054.png', 'fer0002055.png', 'fer0002058.png', 'fer0002064.png', 'fer0002067.png', 'fer0002074.png', 'fer0002076.png', 'fer0002077.png', 'fer0002081.png', 'fer0002082.png', 'fer0002085.png', 'fer0002090.png', 'fer0002099.png', 'fer0002103.png', 'fer0002112.png', 'fer0002129.png', 'fer0002143.png', 'fer0002148.png', 'fer0002149.png', 'fer0002154.png', 'fer0002160.png', 'fer0002162.png', 'fer0002170.png', 'fer0002185.png', 'fer0002187.png', 'fer0002190.png', 'fer0002231.png', 'fer0002232.png', 'fer0002233.png', 'fer0002238.png', 'fer0002240.png', 'fer0002243.png', 'fer0002246.png', 'fer0002251.png', 'fer0002252.png', 'fer0002256.png', 'fer0002258.png', 'fer0002266.png', 'fer0002285.png', 'fer0002297.png', 'fer0002312.png', 'fer0002329.png', 'fer0002333.png', 'fer0002334.png', 'fer0002343.png', 'fer0002353.png', 'fer0002356.png', 'fer0002365.png', 'fer0002368.png', 'fer0002376.png', 'fer0002382.png', 'fer0002384.png', 'fer0002392.png', 'fer0002394.png', 'fer0002395.png', 'fer0002396.png', 'fer0002400.png', 'fer0002408.png', 'fer0002409.png', 'fer0002412.png', 'fer0002421.png', 'fer0002423.png', 'fer0002442.png', 'fer0002451.png', 'fer0002463.png', 'fer0002478.png', 'fer0002480.png', 'fer0002492.png', 'fer0002493.png', 'fer0002510.png', 'fer0002513.png', 'fer0002514.png', 'fer0002522.png', 'fer0002531.png', 'fer0002536.png', 'fer0002547.png', 'fer0002548.png', 'fer0002551.png', 'fer0002554.png', 'fer0002556.png', 'fer0002562.png', 'fer0002572.png', 'fer0002584.png', 'fer0002585.png', 'fer0002595.png', 'fer0002600.png', 'fer0002607.png', 'fer0002613.png', 'fer0002619.png', 'fer0002621.png', 'fer0002623.png', 'fer0002624.png', 'fer0002632.png', 'fer0002640.png', 'fer0002650.png', 'fer0002651.png', 'fer0002658.png', 'fer0002664.png', 'fer0002668.png', 'fer0002675.png', 'fer0002678.png', 'fer0002688.png', 'fer0002692.png', 'fer0002699.png', 'fer0002703.png', 'fer0002713.png', 'fer0002724.png', 'fer0002728.png', 'fer0002737.png', 'fer0002747.png', 'fer0002758.png', 'fer0002791.png', 'fer0002792.png', 'fer0002798.png', 'fer0002799.png', 'fer0002805.png', 'fer0002817.png', 'fer0002820.png', 'fer0002834.png', 'fer0002838.png', 'fer0002847.png', 'fer0002850.png', 'fer0002854.png', 'fer0002869.png', 'fer0002874.png', 'fer0002887.png', 'fer0002901.png', 'fer0002929.png', 'fer0002931.png', 'fer0002965.png', 'fer0002975.png', 'fer0003012.png', 'fer0003018.png', 'fer0003026.png', 'fer0003028.png', 'fer0003039.png', 'fer0003042.png', 'fer0003050.png', 'fer0003060.png', 'fer0003061.png', 'fer0003063.png', 'fer0003070.png', 'fer0003097.png', 'fer0003106.png', 'fer0003107.png', 'fer0003123.png', 'fer0003129.png', 'fer0003132.png', 'fer0003138.png', 'fer0003154.png', 'fer0003162.png', 'fer0003168.png', 'fer0003181.png', 'fer0003193.png', 'fer0003200.png', 'fer0003203.png', 'fer0003208.png', 'fer0003210.png', 'fer0003219.png', 'fer0003223.png', 'fer0003225.png', 'fer0003250.png', 'fer0003257.png', 'fer0003271.png', 'fer0003290.png', 'fer0003291.png', 'fer0003297.png', 'fer0003298.png', 'fer0003300.png', 'fer0003307.png', 'fer0003310.png', 'fer0003312.png', 'fer0003314.png', 'fer0003325.png', 'fer0003329.png', 'fer0003342.png', 'fer0003346.png', 'fer0003354.png', 'fer0003356.png', 'fer0003362.png', 'fer0003364.png', 'fer0003370.png', 'fer0003410.png', 'fer0003418.png', 'fer0003422.png', 'fer0003423.png', 'fer0003424.png', 'fer0003428.png', 'fer0003431.png', 'fer0003432.png', 'fer0003442.png', 'fer0003450.png', 'fer0003456.png', 'fer0003463.png', 'fer0003469.png', 'fer0003488.png', 'fer0003497.png', 'fer0003500.png', 'fer0003507.png', 'fer0003514.png', 'fer0003515.png', 'fer0003535.png', 'fer0003538.png', 'fer0003558.png', 'fer0003559.png', 'fer0003568.png', 'fer0003574.png', 'fer0003587.png', 'fer0003605.png', 'fer0003629.png', 'fer0003647.png', 'fer0003690.png', 'fer0003699.png', 'fer0003717.png', 'fer0003729.png', 'fer0003734.png', 'fer0003735.png', 'fer0003746.png', 'fer0003752.png', 'fer0003756.png', 'fer0003765.png', 'fer0003766.png', 'fer0003771.png', 'fer0003791.png', 'fer0003792.png', 'fer0003795.png', 'fer0003802.png', 'fer0003804.png', 'fer0003814.png', 'fer0003815.png', 'fer0003829.png', 'fer0003832.png', 'fer0003833.png', 'fer0003838.png', 'fer0003848.png', 'fer0003858.png', 'fer0003862.png', 'fer0003872.png', 'fer0003874.png', 'fer0003876.png', 'fer0003892.png', 'fer0003894.png', 'fer0003897.png', 'fer0003908.png', 'fer0003915.png', 'fer0003921.png', 'fer0003935.png', 'fer0003944.png', 'fer0003968.png', 'fer0003981.png', 'fer0003987.png', 'fer0004000.png', 'fer0004001.png', 'fer0004012.png', 'fer0004015.png', 'fer0004033.png', 'fer0004040.png', 'fer0004042.png', 'fer0004067.png', 'fer0004070.png', 'fer0004075.png', 'fer0004082.png', 'fer0004091.png', 'fer0004093.png', 'fer0004096.png', 'fer0004117.png', 'fer0004124.png', 'fer0004125.png', 'fer0004129.png', 'fer0004134.png', 'fer0004136.png', 'fer0004147.png', 'fer0004160.png', 'fer0004170.png', 'fer0004192.png', 'fer0004196.png', 'fer0004200.png', 'fer0004201.png', 'fer0004218.png', 'fer0004228.png', 'fer0004230.png', 'fer0004232.png', 'fer0004235.png', 'fer0004254.png', 'fer0004260.png', 'fer0004267.png', 'fer0004288.png', 'fer0004310.png', 'fer0004327.png', 'fer0004332.png', 'fer0004344.png', 'fer0004350.png', 'fer0004361.png', 'fer0004365.png', 'fer0004366.png', 'fer0004373.png', 'fer0004383.png', 'fer0004392.png', 'fer0004394.png', 'fer0004397.png', 'fer0004402.png', 'fer0004412.png', 'fer0004423.png', 'fer0004439.png', 'fer0004440.png', 'fer0004442.png', 'fer0004444.png', 'fer0004448.png', 'fer0004464.png', 'fer0004471.png', 'fer0004488.png', 'fer0004491.png', 'fer0004492.png', 'fer0004494.png', 'fer0004502.png', 'fer0004515.png', 'fer0004530.png', 'fer0004533.png', 'fer0004559.png', 'fer0004561.png', 'fer0004579.png', 'fer0004582.png', 'fer0004583.png', 'fer0004600.png', 'fer0004601.png', 'fer0004605.png', 'fer0004620.png', 'fer0004627.png', 'fer0004628.png', 'fer0004630.png', 'fer0004633.png', 'fer0004637.png', 'fer0004647.png', 'fer0004648.png', 'fer0004666.png', 'fer0004670.png', 'fer0004671.png', 'fer0004672.png', 'fer0004673.png', 'fer0004676.png', 'fer0004701.png', 'fer0004707.png', 'fer0004709.png', 'fer0004710.png', 'fer0004725.png', 'fer0004748.png', 'fer0004749.png', 'fer0004750.png', 'fer0004753.png', 'fer0004755.png', 'fer0004768.png', 'fer0004795.png', 'fer0004797.png', 'fer0004803.png', 'fer0004809.png', 'fer0004844.png', 'fer0004845.png', 'fer0004849.png', 'fer0004858.png', 'fer0004862.png', 'fer0004864.png', 'fer0004866.png', 'fer0004874.png', 'fer0004881.png', 'fer0004888.png', 'fer0004894.png', 'fer0004908.png', 'fer0004913.png', 'fer0004919.png', 'fer0004922.png', 'fer0004935.png', 'fer0004938.png', 'fer0004939.png', 'fer0004941.png', 'fer0004948.png', 'fer0004953.png', 'fer0004955.png', 'fer0004964.png', 'fer0004968.png', 'fer0004969.png', 'fer0004973.png', 'fer0004982.png', 'fer0004984.png', 'fer0004989.png', 'fer0005024.png', 'fer0005031.png', 'fer0005037.png', 'fer0005051.png', 'fer0005052.png', 'fer0005053.png', 'fer0005057.png', 'fer0005060.png', 'fer0005083.png', 'fer0005087.png', 'fer0005113.png', 'fer0005116.png', 'fer0005123.png', 'fer0005149.png', 'fer0005168.png', 'fer0005180.png', 'fer0005186.png', 'fer0005192.png', 'fer0005193.png', 'fer0005203.png', 'fer0005204.png', 'fer0005215.png', 'fer0005222.png', 'fer0005223.png', 'fer0005234.png', 'fer0005239.png', 'fer0005253.png', 'fer0005276.png', 'fer0005277.png', 'fer0005283.png', 'fer0005288.png', 'fer0005291.png', 'fer0005294.png', 'fer0005297.png', 'fer0005310.png', 'fer0005314.png', 'fer0005317.png', 'fer0005333.png', 'fer0005337.png', 'fer0005368.png', 'fer0005370.png', 'fer0005375.png', 'fer0005377.png', 'fer0005406.png', 'fer0005414.png', 'fer0005423.png', 'fer0005426.png', 'fer0005435.png', 'fer0005461.png', 'fer0005463.png', 'fer0005474.png', 'fer0005481.png', 'fer0005482.png', 'fer0005483.png', 'fer0005484.png', 'fer0005494.png', 'fer0005499.png', 'fer0005506.png', 'fer0005509.png', 'fer0005510.png', 'fer0005512.png', 'fer0005513.png', 'fer0005518.png', 'fer0005522.png', 'fer0005555.png', 'fer0005566.png', 'fer0005570.png', 'fer0005573.png', 'fer0005577.png', 'fer0005581.png', 'fer0005596.png', 'fer0005635.png', 'fer0005648.png', 'fer0005654.png', 'fer0005666.png', 'fer0005671.png', 'fer0005681.png', 'fer0005682.png', 'fer0005690.png', 'fer0005694.png', 'fer0005700.png', 'fer0005714.png', 'fer0005732.png', 'fer0005738.png', 'fer0005768.png', 'fer0005777.png', 'fer0005779.png', 'fer0005784.png', 'fer0005790.png', 'fer0005810.png', 'fer0005832.png', 'fer0005836.png', 'fer0005841.png', 'fer0005843.png', 'fer0005844.png', 'fer0005846.png', 'fer0005847.png', 'fer0005851.png', 'fer0005852.png', 'fer0005860.png', 'fer0005863.png', 'fer0005870.png', 'fer0005875.png', 'fer0005879.png', 'fer0005881.png', 'fer0005884.png', 'fer0005889.png', 'fer0005894.png', 'fer0005899.png', 'fer0005905.png', 'fer0005909.png', 'fer0005911.png', 'fer0005912.png', 'fer0005917.png', 'fer0005918.png', 'fer0005921.png', 'fer0005932.png', 'fer0005934.png', 'fer0005937.png', 'fer0005944.png', 'fer0005951.png', 'fer0005954.png', 'fer0005962.png', 'fer0005967.png', 'fer0005979.png', 'fer0005981.png', 'fer0005986.png', 'fer0005989.png', 'fer0005999.png', 'fer0006010.png', 'fer0006036.png', 'fer0006049.png', 'fer0006055.png', 'fer0006077.png', 'fer0006086.png', 'fer0006094.png', 'fer0006099.png', 'fer0006102.png', 'fer0006115.png', 'fer0006116.png', 'fer0006119.png', 'fer0006132.png', 'fer0006188.png', 'fer0006192.png', 'fer0006195.png', 'fer0006208.png', 'fer0006214.png', 'fer0006215.png', 'fer0006229.png', 'fer0006232.png', 'fer0006256.png', 'fer0006259.png', 'fer0006297.png', 'fer0006298.png', 'fer0006303.png', 'fer0006307.png', 'fer0006311.png', 'fer0006317.png', 'fer0006327.png', 'fer0006334.png', 'fer0006340.png', 'fer0006351.png', 'fer0006352.png', 'fer0006372.png', 'fer0006377.png', 'fer0006378.png', 'fer0006395.png', 'fer0006397.png', 'fer0006400.png', 'fer0006408.png', 'fer0006411.png', 'fer0006413.png', 'fer0006418.png', 'fer0006435.png', 'fer0006447.png', 'fer0006451.png', 'fer0006471.png', 'fer0006477.png', 'fer0006478.png', 'fer0006482.png', 'fer0006498.png', 'fer0006503.png', 'fer0006518.png', 'fer0006533.png', 'fer0006534.png', 'fer0006542.png', 'fer0006551.png', 'fer0006560.png', 'fer0006566.png', 'fer0006567.png', 'fer0006577.png', 'fer0006606.png', 'fer0006607.png', 'fer0006608.png', 'fer0006625.png', 'fer0006628.png', 'fer0006639.png', 'fer0006642.png', 'fer0006648.png', 'fer0006653.png', 'fer0006655.png', 'fer0006659.png', 'fer0006706.png', 'fer0006708.png', 'fer0006713.png', 'fer0006725.png', 'fer0006731.png', 'fer0006734.png', 'fer0006735.png', 'fer0006737.png', 'fer0006739.png', 'fer0006742.png', 'fer0006753.png', 'fer0006756.png', 'fer0006761.png', 'fer0006763.png', 'fer0006765.png', 'fer0006771.png', 'fer0006789.png', 'fer0006799.png', 'fer0006801.png', 'fer0006805.png', 'fer0006819.png', 'fer0006830.png', 'fer0006835.png', 'fer0006838.png', 'fer0006845.png', 'fer0006852.png', 'fer0006855.png', 'fer0006862.png', 'fer0006863.png', 'fer0006867.png', 'fer0006887.png', 'fer0006892.png', 'fer0006907.png', 'fer0006908.png', 'fer0006919.png', 'fer0006924.png', 'fer0006926.png', 'fer0006927.png', 'fer0006934.png', 'fer0006942.png', 'fer0006949.png', 'fer0006953.png', 'fer0006955.png', 'fer0006974.png', 'fer0006980.png', 'fer0006989.png', 'fer0007004.png', 'fer0007006.png', 'fer0007014.png', 'fer0007024.png', 'fer0007025.png', 'fer0007041.png', 'fer0007052.png', 'fer0007081.png', 'fer0007086.png', 'fer0007093.png', 'fer0007094.png', 'fer0007100.png', 'fer0007118.png', 'fer0007124.png', 'fer0007145.png', 'fer0007152.png', 'fer0007153.png', 'fer0007166.png', 'fer0007191.png', 'fer0007200.png', 'fer0007226.png', 'fer0007231.png', 'fer0007236.png', 'fer0007256.png', 'fer0007269.png', 'fer0007283.png', 'fer0007287.png', 'fer0007297.png', 'fer0007311.png', 'fer0007321.png', 'fer0007322.png', 'fer0007323.png', 'fer0007325.png', 'fer0007326.png', 'fer0007328.png', 'fer0007337.png', 'fer0007339.png', 'fer0007343.png', 'fer0007345.png', 'fer0007346.png', 'fer0007348.png', 'fer0007354.png', 'fer0007366.png', 'fer0007368.png', 'fer0007375.png', 'fer0007389.png', 'fer0007393.png', 'fer0007402.png', 'fer0007417.png', 'fer0007419.png', 'fer0007433.png', 'fer0007446.png', 'fer0007455.png', 'fer0007468.png', 'fer0007474.png', 'fer0007482.png', 'fer0007484.png', 'fer0007488.png', 'fer0007491.png', 'fer0007493.png', 'fer0007499.png', 'fer0007503.png', 'fer0007531.png', 'fer0007536.png', 'fer0007541.png', 'fer0007544.png', 'fer0007545.png', 'fer0007549.png', 'fer0007556.png', 'fer0007561.png', 'fer0007588.png', 'fer0007630.png', 'fer0007637.png', 'fer0007644.png', 'fer0007659.png', 'fer0007668.png', 'fer0007669.png', 'fer0007672.png', 'fer0007678.png', 'fer0007685.png', 'fer0007696.png', 'fer0007699.png', 'fer0007701.png', 'fer0007703.png', 'fer0007710.png', 'fer0007713.png', 'fer0007730.png', 'fer0007745.png', 'fer0007768.png', 'fer0007775.png', 'fer0007783.png', 'fer0007788.png', 'fer0007791.png', 'fer0007792.png', 'fer0007799.png', 'fer0007800.png', 'fer0007806.png', 'fer0007819.png', 'fer0007821.png', 'fer0007845.png', 'fer0007850.png', 'fer0007856.png', 'fer0007872.png', 'fer0007880.png', 'fer0007891.png', 'fer0007893.png', 'fer0007898.png', 'fer0007917.png', 'fer0007928.png', 'fer0007932.png', 'fer0007936.png', 'fer0007956.png', 'fer0007963.png', 'fer0007968.png', 'fer0007985.png', 'fer0007998.png', 'fer0008000.png', 'fer0008003.png', 'fer0008005.png', 'fer0008009.png', 'fer0008020.png', 'fer0008023.png', 'fer0008025.png', 'fer0008026.png', 'fer0008054.png', 'fer0008080.png', 'fer0008093.png', 'fer0008096.png', 'fer0008103.png', 'fer0008124.png', 'fer0008128.png', 'fer0008145.png', 'fer0008146.png', 'fer0008177.png', 'fer0008187.png', 'fer0008197.png', 'fer0008222.png', 'fer0008231.png', 'fer0008237.png', 'fer0008238.png', 'fer0008255.png', 'fer0008262.png', 'fer0008284.png', 'fer0008303.png', 'fer0008304.png', 'fer0008306.png', 'fer0008310.png', 'fer0008313.png', 'fer0008320.png', 'fer0008349.png', 'fer0008383.png', 'fer0008397.png', 'fer0008420.png', 'fer0008423.png', 'fer0008435.png', 'fer0008443.png', 'fer0008444.png', 'fer0008446.png', 'fer0008451.png', 'fer0008452.png', 'fer0008455.png', 'fer0008462.png', 'fer0008475.png', 'fer0008479.png', 'fer0008483.png', 'fer0008489.png', 'fer0008497.png', 'fer0008499.png', 'fer0008502.png', 'fer0008510.png', 'fer0008526.png', 'fer0008531.png', 'fer0008532.png', 'fer0008534.png', 'fer0008536.png', 'fer0008542.png', 'fer0008552.png', 'fer0008561.png', 'fer0008563.png', 'fer0008568.png', 'fer0008576.png', 'fer0008578.png', 'fer0008584.png', 'fer0008594.png', 'fer0008596.png', 'fer0008601.png', 'fer0008604.png', 'fer0008605.png', 'fer0008625.png', 'fer0008627.png', 'fer0008666.png', 'fer0008685.png', 'fer0008698.png', 'fer0008699.png', 'fer0008703.png', 'fer0008715.png', 'fer0008722.png', 'fer0008737.png', 'fer0008739.png', 'fer0008743.png', 'fer0008750.png', 'fer0008767.png', 'fer0008768.png', 'fer0008794.png', 'fer0008802.png', 'fer0008805.png', 'fer0008809.png', 'fer0008816.png', 'fer0008818.png', 'fer0008821.png', 'fer0008822.png', 'fer0008825.png', 'fer0008836.png', 'fer0008841.png', 'fer0008855.png', 'fer0008865.png', 'fer0008921.png', 'fer0008926.png', 'fer0008937.png', 'fer0008939.png', 'fer0008945.png', 'fer0008949.png', 'fer0008965.png', 'fer0008967.png', 'fer0008975.png', 'fer0008982.png', 'fer0008984.png', 'fer0008993.png', 'fer0009001.png', 'fer0009007.png', 'fer0009021.png', 'fer0009022.png', 'fer0009024.png', 'fer0009047.png', 'fer0009050.png', 'fer0009063.png', 'fer0009067.png', 'fer0009068.png', 'fer0009072.png', 'fer0009075.png', 'fer0009088.png', 'fer0009091.png', 'fer0009101.png', 'fer0009107.png', 'fer0009109.png', 'fer0009117.png', 'fer0009133.png', 'fer0009134.png', 'fer0009150.png', 'fer0009155.png', 'fer0009158.png', 'fer0009170.png', 'fer0009171.png', 'fer0009177.png', 'fer0009202.png', 'fer0009203.png', 'fer0009208.png', 'fer0009215.png', 'fer0009223.png', 'fer0009226.png', 'fer0009227.png', 'fer0009237.png', 'fer0009242.png', 'fer0009246.png', 'fer0009247.png', 'fer0009251.png', 'fer0009257.png', 'fer0009259.png', 'fer0009298.png', 'fer0009300.png', 'fer0009336.png', 'fer0009352.png', 'fer0009354.png', 'fer0009355.png', 'fer0009360.png', 'fer0009362.png', 'fer0009365.png', 'fer0009383.png', 'fer0009395.png', 'fer0009403.png', 'fer0009405.png', 'fer0009420.png', 'fer0009425.png', 'fer0009426.png', 'fer0009428.png', 'fer0009440.png', 'fer0009447.png', 'fer0009453.png', 'fer0009456.png', 'fer0009465.png', 'fer0009480.png', 'fer0009481.png', 'fer0009486.png', 'fer0009491.png', 'fer0009522.png', 'fer0009534.png', 'fer0009544.png', 'fer0009554.png', 'fer0009561.png', 'fer0009569.png', 'fer0009570.png', 'fer0009572.png', 'fer0009574.png', 'fer0009580.png', 'fer0009589.png', 'fer0009592.png', 'fer0009595.png', 'fer0009596.png', 'fer0009600.png', 'fer0009608.png', 'fer0009616.png', 'fer0009619.png', 'fer0009622.png', 'fer0009627.png', 'fer0009641.png', 'fer0009643.png', 'fer0009659.png', 'fer0009682.png', 'fer0009689.png', 'fer0009705.png', 'fer0009712.png', 'fer0009722.png', 'fer0009734.png', 'fer0009739.png', 'fer0009743.png', 'fer0009751.png', 'fer0009759.png', 'fer0009762.png', 'fer0009766.png', 'fer0009773.png', 'fer0009781.png', 'fer0009798.png', 'fer0009807.png', 'fer0009830.png', 'fer0009833.png', 'fer0009837.png', 'fer0009843.png', 'fer0009845.png', 'fer0009858.png', 'fer0009860.png', 'fer0009863.png', 'fer0009865.png', 'fer0009878.png', 'fer0009886.png', 'fer0009906.png', 'fer0009909.png', 'fer0009920.png', 'fer0009924.png', 'fer0009926.png', 'fer0009928.png', 'fer0009941.png', 'fer0009967.png', 'fer0009977.png', 'fer0009992.png', 'fer0009995.png', 'fer0010001.png', 'fer0010003.png', 'fer0010015.png', 'fer0010025.png', 'fer0010031.png', 'fer0010041.png', 'fer0010043.png', 'fer0010045.png', 'fer0010060.png', 'fer0010089.png', 'fer0010098.png', 'fer0010099.png', 'fer0010102.png', 'fer0010103.png', 'fer0010113.png', 'fer0010115.png', 'fer0010116.png', 'fer0010121.png', 'fer0010124.png', 'fer0010137.png', 'fer0010161.png', 'fer0010174.png', 'fer0010178.png', 'fer0010182.png', 'fer0010183.png', 'fer0010185.png', 'fer0010191.png', 'fer0010200.png', 'fer0010202.png', 'fer0010203.png', 'fer0010209.png', 'fer0010230.png', 'fer0010244.png', 'fer0010251.png', 'fer0010272.png', 'fer0010276.png', 'fer0010285.png', 'fer0010317.png', 'fer0010336.png', 'fer0010341.png', 'fer0010343.png', 'fer0010357.png', 'fer0010358.png', 'fer0010367.png', 'fer0010369.png', 'fer0010385.png', 'fer0010392.png', 'fer0010410.png', 'fer0010419.png', 'fer0010425.png', 'fer0010426.png', 'fer0010427.png', 'fer0010430.png', 'fer0010432.png', 'fer0010435.png', 'fer0010439.png', 'fer0010445.png', 'fer0010451.png', 'fer0010460.png', 'fer0010468.png', 'fer0010492.png', 'fer0010510.png', 'fer0010522.png', 'fer0010532.png', 'fer0010547.png', 'fer0010551.png', 'fer0010553.png', 'fer0010559.png', 'fer0010560.png', 'fer0010569.png', 'fer0010577.png', 'fer0010578.png', 'fer0010582.png', 'fer0010588.png', 'fer0010599.png', 'fer0010610.png', 'fer0010619.png', 'fer0010628.png', 'fer0010642.png', 'fer0010671.png', 'fer0010698.png', 'fer0010710.png', 'fer0010711.png', 'fer0010718.png', 'fer0010727.png', 'fer0010734.png', 'fer0010746.png', 'fer0010751.png', 'fer0010758.png', 'fer0010777.png', 'fer0010779.png', 'fer0010788.png', 'fer0010790.png', 'fer0010791.png', 'fer0010793.png', 'fer0010798.png', 'fer0010804.png', 'fer0010806.png', 'fer0010809.png', 'fer0010818.png', 'fer0010822.png', 'fer0010824.png', 'fer0010825.png', 'fer0010840.png', 'fer0010842.png', 'fer0010849.png', 'fer0010853.png', 'fer0010855.png', 'fer0010862.png', 'fer0010868.png', 'fer0010875.png', 'fer0010883.png', 'fer0010886.png', 'fer0010888.png', 'fer0010889.png', 'fer0010895.png', 'fer0010896.png', 'fer0010904.png', 'fer0010910.png', 'fer0010916.png', 'fer0010920.png', 'fer0010923.png', 'fer0010936.png', 'fer0010956.png', 'fer0010959.png', 'fer0010970.png', 'fer0010975.png', 'fer0010980.png', 'fer0010981.png', 'fer0010989.png', 'fer0011019.png', 'fer0011020.png', 'fer0011021.png', 'fer0011024.png', 'fer0011033.png', 'fer0011039.png', 'fer0011045.png', 'fer0011051.png', 'fer0011059.png', 'fer0011067.png', 'fer0011070.png', 'fer0011105.png', 'fer0011116.png', 'fer0011120.png', 'fer0011122.png', 'fer0011140.png', 'fer0011146.png', 'fer0011147.png', 'fer0011159.png', 'fer0011171.png', 'fer0011177.png', 'fer0011180.png', 'fer0011181.png', 'fer0011200.png', 'fer0011203.png', 'fer0011211.png', 'fer0011214.png', 'fer0011219.png', 'fer0011232.png', 'fer0011235.png', 'fer0011244.png', 'fer0011250.png', 'fer0011252.png', 'fer0011261.png', 'fer0011269.png', 'fer0011278.png', 'fer0011283.png', 'fer0011288.png', 'fer0011322.png', 'fer0011325.png', 'fer0011331.png', 'fer0011332.png', 'fer0011346.png', 'fer0011351.png', 'fer0011352.png', 'fer0011357.png', 'fer0011362.png', 'fer0011364.png', 'fer0011369.png', 'fer0011370.png', 'fer0011376.png', 'fer0011381.png', 'fer0011384.png', 'fer0011386.png', 'fer0011397.png', 'fer0011399.png', 'fer0011402.png', 'fer0011403.png', 'fer0011404.png', 'fer0011411.png', 'fer0011424.png', 'fer0011436.png', 'fer0011437.png', 'fer0011439.png', 'fer0011448.png', 'fer0011455.png', 'fer0011463.png', 'fer0011465.png', 'fer0011480.png', 'fer0011481.png', 'fer0011484.png', 'fer0011489.png', 'fer0011491.png', 'fer0011492.png', 'fer0011493.png', 'fer0011494.png', 'fer0011505.png', 'fer0011513.png', 'fer0011515.png', 'fer0011517.png', 'fer0011523.png', 'fer0011525.png', 'fer0011570.png', 'fer0011608.png', 'fer0011621.png', 'fer0011642.png', 'fer0011646.png', 'fer0011647.png', 'fer0011668.png', 'fer0011669.png', 'fer0011678.png', 'fer0011705.png', 'fer0011706.png', 'fer0011709.png', 'fer0011712.png', 'fer0011713.png', 'fer0011717.png', 'fer0011725.png', 'fer0011727.png', 'fer0011730.png', 'fer0011737.png', 'fer0011748.png', 'fer0011757.png', 'fer0011758.png', 'fer0011772.png', 'fer0011791.png', 'fer0011800.png', 'fer0011809.png', 'fer0011811.png', 'fer0011824.png', 'fer0011828.png', 'fer0011832.png', 'fer0011844.png', 'fer0011850.png', 'fer0011859.png', 'fer0011876.png', 'fer0011884.png', 'fer0011886.png', 'fer0011888.png', 'fer0011890.png', 'fer0011896.png', 'fer0011907.png', 'fer0011915.png', 'fer0011935.png', 'fer0011938.png', 'fer0011947.png', 'fer0011948.png', 'fer0011951.png', 'fer0011987.png', 'fer0012001.png', 'fer0012003.png', 'fer0012010.png', 'fer0012016.png', 'fer0012021.png', 'fer0012025.png', 'fer0012032.png', 'fer0012053.png', 'fer0012075.png', 'fer0012082.png', 'fer0012083.png', 'fer0012106.png', 'fer0012113.png', 'fer0012115.png', 'fer0012134.png', 'fer0012184.png', 'fer0012186.png', 'fer0012192.png', 'fer0012205.png', 'fer0012216.png', 'fer0012228.png', 'fer0012230.png', 'fer0012243.png', 'fer0012248.png', 'fer0012250.png', 'fer0012258.png', 'fer0012259.png', 'fer0012271.png', 'fer0012278.png', 'fer0012279.png', 'fer0012283.png', 'fer0012285.png', 'fer0012308.png', 'fer0012311.png', 'fer0012312.png', 'fer0012321.png', 'fer0012327.png', 'fer0012359.png', 'fer0012365.png', 'fer0012382.png', 'fer0012389.png', 'fer0012392.png', 'fer0012397.png', 'fer0012420.png', 'fer0012428.png', 'fer0012435.png', 'fer0012463.png', 'fer0012464.png', 'fer0012468.png', 'fer0012484.png', 'fer0012499.png', 'fer0012505.png', 'fer0012506.png', 'fer0012511.png', 'fer0012517.png', 'fer0012523.png', 'fer0012528.png', 'fer0012537.png', 'fer0012567.png', 'fer0012580.png', 'fer0012589.png', 'fer0012590.png', 'fer0012602.png', 'fer0012606.png', 'fer0012621.png', 'fer0012625.png', 'fer0012627.png', 'fer0012628.png', 'fer0012632.png', 'fer0012641.png', 'fer0012645.png', 'fer0012657.png', 'fer0012667.png', 'fer0012676.png', 'fer0012679.png', 'fer0012680.png', 'fer0012681.png', 'fer0012691.png', 'fer0012695.png', 'fer0012711.png', 'fer0012717.png', 'fer0012718.png', 'fer0012725.png', 'fer0012730.png', 'fer0012756.png', 'fer0012760.png', 'fer0012764.png', 'fer0012777.png', 'fer0012778.png', 'fer0012790.png', 'fer0012791.png', 'fer0012794.png', 'fer0012799.png', 'fer0012800.png', 'fer0012816.png', 'fer0012818.png', 'fer0012819.png', 'fer0012831.png', 'fer0012835.png', 'fer0012841.png', 'fer0012856.png', 'fer0012864.png', 'fer0012870.png', 'fer0012874.png', 'fer0012881.png', 'fer0012888.png', 'fer0012895.png', 'fer0012900.png', 'fer0012902.png', 'fer0012910.png', 'fer0012912.png', 'fer0012920.png', 'fer0012924.png', 'fer0012930.png', 'fer0012945.png', 'fer0012951.png', 'fer0012959.png', 'fer0012963.png', 'fer0012965.png', 'fer0012967.png', 'fer0012978.png', 'fer0012986.png', 'fer0012992.png', 'fer0012993.png', 'fer0012994.png', 'fer0012995.png', 'fer0013001.png', 'fer0013003.png', 'fer0013013.png', 'fer0013022.png', 'fer0013029.png', 'fer0013030.png', 'fer0013036.png', 'fer0013043.png', 'fer0013050.png', 'fer0013058.png', 'fer0013060.png', 'fer0013091.png', 'fer0013093.png', 'fer0013106.png', 'fer0013111.png', 'fer0013118.png', 'fer0013124.png', 'fer0013135.png', 'fer0013136.png', 'fer0013144.png', 'fer0013146.png', 'fer0013149.png', 'fer0013162.png', 'fer0013165.png', 'fer0013173.png', 'fer0013191.png', 'fer0013194.png', 'fer0013200.png', 'fer0013201.png', 'fer0013203.png', 'fer0013213.png', 'fer0013222.png', 'fer0013228.png', 'fer0013238.png', 'fer0013242.png', 'fer0013260.png', 'fer0013263.png', 'fer0013269.png', 'fer0013270.png', 'fer0013279.png', 'fer0013283.png', 'fer0013289.png', 'fer0013309.png', 'fer0013313.png', 'fer0013320.png', 'fer0013323.png', 'fer0013326.png', 'fer0013328.png', 'fer0013336.png', 'fer0013338.png', 'fer0013342.png', 'fer0013345.png', 'fer0013354.png', 'fer0013371.png', 'fer0013372.png', 'fer0013379.png', 'fer0013389.png', 'fer0013392.png', 'fer0013401.png', 'fer0013405.png', 'fer0013414.png', 'fer0013421.png', 'fer0013429.png', 'fer0013435.png', 'fer0013441.png', 'fer0013449.png', 'fer0013452.png', 'fer0013457.png', 'fer0013466.png', 'fer0013469.png', 'fer0013480.png', 'fer0013483.png', 'fer0013503.png', 'fer0013504.png', 'fer0013508.png', 'fer0013514.png', 'fer0013518.png', 'fer0013520.png', 'fer0013525.png', 'fer0013534.png', 'fer0013545.png', 'fer0013546.png', 'fer0013550.png', 'fer0013597.png', 'fer0013610.png', 'fer0013615.png', 'fer0013617.png', 'fer0013619.png', 'fer0013654.png', 'fer0013666.png', 'fer0013685.png', 'fer0013698.png', 'fer0013709.png', 'fer0013739.png', 'fer0013741.png', 'fer0013754.png', 'fer0013768.png', 'fer0013788.png', 'fer0013794.png', 'fer0013801.png', 'fer0013807.png', 'fer0013814.png', 'fer0013817.png', 'fer0013819.png', 'fer0013827.png', 'fer0013845.png', 'fer0013849.png', 'fer0013868.png', 'fer0013871.png', 'fer0013872.png', 'fer0013875.png', 'fer0013899.png', 'fer0013908.png', 'fer0013909.png', 'fer0013917.png', 'fer0013929.png', 'fer0013933.png', 'fer0013940.png', 'fer0013943.png', 'fer0013949.png', 'fer0013971.png', 'fer0013975.png', 'fer0013978.png', 'fer0014004.png', 'fer0014014.png', 'fer0014018.png', 'fer0014022.png', 'fer0014024.png', 'fer0014026.png', 'fer0014044.png', 'fer0014052.png', 'fer0014053.png', 'fer0014059.png', 'fer0014075.png', 'fer0014078.png', 'fer0014081.png', 'fer0014089.png', 'fer0014092.png', 'fer0014097.png', 'fer0014103.png', 'fer0014104.png', 'fer0014145.png', 'fer0014152.png', 'fer0014153.png', 'fer0014154.png', 'fer0014158.png', 'fer0014162.png', 'fer0014167.png', 'fer0014168.png', 'fer0014180.png', 'fer0014181.png', 'fer0014191.png', 'fer0014195.png', 'fer0014207.png', 'fer0014239.png', 'fer0014262.png', 'fer0014272.png', 'fer0014296.png', 'fer0014297.png', 'fer0014298.png', 'fer0014300.png', 'fer0014303.png', 'fer0014314.png', 'fer0014332.png', 'fer0014339.png', 'fer0014342.png', 'fer0014343.png', 'fer0014352.png', 'fer0014356.png', 'fer0014369.png', 'fer0014376.png', 'fer0014397.png', 'fer0014398.png', 'fer0014408.png', 'fer0014416.png', 'fer0014421.png', 'fer0014423.png', 'fer0014427.png', 'fer0014430.png', 'fer0014431.png', 'fer0014435.png', 'fer0014436.png', 'fer0014437.png', 'fer0014450.png', 'fer0014452.png', 'fer0014457.png', 'fer0014463.png', 'fer0014464.png', 'fer0014466.png', 'fer0014471.png', 'fer0014473.png', 'fer0014476.png', 'fer0014478.png', 'fer0014482.png', 'fer0014487.png', 'fer0014503.png', 'fer0014520.png', 'fer0014529.png', 'fer0014535.png', 'fer0014536.png', 'fer0014564.png', 'fer0014567.png', 'fer0014572.png', 'fer0014573.png', 'fer0014591.png', 'fer0014609.png', 'fer0014615.png', 'fer0014616.png', 'fer0014628.png', 'fer0014630.png', 'fer0014633.png', 'fer0014639.png', 'fer0014657.png', 'fer0014680.png', 'fer0014699.png', 'fer0014702.png', 'fer0014703.png', 'fer0014715.png', 'fer0014735.png', 'fer0014748.png', 'fer0014749.png', 'fer0014763.png', 'fer0014802.png', 'fer0014805.png', 'fer0014807.png', 'fer0014820.png', 'fer0014821.png', 'fer0014823.png', 'fer0014827.png', 'fer0014845.png', 'fer0014860.png', 'fer0014865.png', 'fer0014891.png', 'fer0014899.png', 'fer0014901.png', 'fer0014920.png', 'fer0014925.png', 'fer0014931.png', 'fer0014942.png', 'fer0014943.png', 'fer0014947.png', 'fer0014952.png', 'fer0014958.png', 'fer0014961.png', 'fer0014969.png', 'fer0014971.png', 'fer0014978.png', 'fer0014982.png', 'fer0014985.png', 'fer0015004.png', 'fer0015005.png', 'fer0015008.png', 'fer0015010.png', 'fer0015012.png', 'fer0015014.png', 'fer0015023.png', 'fer0015032.png', 'fer0015033.png', 'fer0015038.png', 'fer0015043.png', 'fer0015045.png', 'fer0015052.png', 'fer0015053.png', 'fer0015055.png', 'fer0015062.png', 'fer0015064.png', 'fer0015069.png', 'fer0015070.png', 'fer0015101.png', 'fer0015102.png', 'fer0015109.png', 'fer0015110.png', 'fer0015112.png', 'fer0015122.png', 'fer0015132.png', 'fer0015137.png', 'fer0015149.png', 'fer0015156.png', 'fer0015161.png', 'fer0015166.png', 'fer0015173.png', 'fer0015176.png', 'fer0015177.png', 'fer0015181.png', 'fer0015191.png', 'fer0015203.png', 'fer0015209.png', 'fer0015222.png', 'fer0015233.png', 'fer0015255.png', 'fer0015270.png', 'fer0015290.png', 'fer0015293.png', 'fer0015297.png', 'fer0015298.png', 'fer0015311.png', 'fer0015317.png', 'fer0015321.png', 'fer0015325.png', 'fer0015328.png', 'fer0015330.png', 'fer0015347.png', 'fer0015358.png', 'fer0015363.png', 'fer0015373.png', 'fer0015384.png', 'fer0015388.png', 'fer0015394.png', 'fer0015411.png', 'fer0015415.png', 'fer0015432.png', 'fer0015445.png', 'fer0015446.png', 'fer0015448.png', 'fer0015487.png', 'fer0015496.png', 'fer0015504.png', 'fer0015517.png', 'fer0015523.png', 'fer0015532.png', 'fer0015536.png', 'fer0015550.png', 'fer0015551.png', 'fer0015564.png', 'fer0015566.png', 'fer0015569.png', 'fer0015571.png', 'fer0015574.png', 'fer0015589.png', 'fer0015595.png', 'fer0015611.png', 'fer0015612.png', 'fer0015613.png', 'fer0015628.png', 'fer0015630.png', 'fer0015650.png', 'fer0015675.png', 'fer0015681.png', 'fer0015695.png', 'fer0015700.png', 'fer0015717.png', 'fer0015726.png', 'fer0015737.png', 'fer0015738.png', 'fer0015747.png', 'fer0015767.png', 'fer0015769.png', 'fer0015782.png', 'fer0015789.png', 'fer0015792.png', 'fer0015793.png', 'fer0015797.png', 'fer0015799.png', 'fer0015800.png', 'fer0015801.png', 'fer0015808.png', 'fer0015811.png', 'fer0015816.png', 'fer0015824.png', 'fer0015841.png', 'fer0015845.png', 'fer0015850.png', 'fer0015864.png', 'fer0015870.png', 'fer0015878.png', 'fer0015882.png', 'fer0015884.png', 'fer0015889.png', 'fer0015897.png', 'fer0015899.png', 'fer0015903.png', 'fer0015909.png', 'fer0015918.png', 'fer0015935.png', 'fer0015966.png', 'fer0015967.png', 'fer0015977.png', 'fer0015978.png', 'fer0015983.png', 'fer0016008.png', 'fer0016009.png', 'fer0016016.png', 'fer0016029.png', 'fer0016043.png', 'fer0016053.png', 'fer0016066.png', 'fer0016067.png', 'fer0016074.png', 'fer0016093.png', 'fer0016102.png', 'fer0016107.png', 'fer0016115.png', 'fer0016129.png', 'fer0016132.png', 'fer0016138.png', 'fer0016150.png', 'fer0016155.png', 'fer0016158.png', 'fer0016163.png', 'fer0016188.png', 'fer0016189.png', 'fer0016195.png', 'fer0016199.png', 'fer0016203.png', 'fer0016204.png', 'fer0016206.png', 'fer0016207.png', 'fer0016209.png', 'fer0016219.png', 'fer0016222.png', 'fer0016224.png', 'fer0016229.png', 'fer0016235.png', 'fer0016236.png', 'fer0016241.png', 'fer0016245.png', 'fer0016266.png', 'fer0016271.png', 'fer0016303.png', 'fer0016306.png', 'fer0016311.png', 'fer0016314.png', 'fer0016317.png', 'fer0016328.png', 'fer0016332.png', 'fer0016334.png', 'fer0016335.png', 'fer0016346.png', 'fer0016353.png', 'fer0016358.png', 'fer0016372.png', 'fer0016390.png', 'fer0016392.png', 'fer0016396.png', 'fer0016399.png', 'fer0016423.png', 'fer0016430.png', 'fer0016433.png', 'fer0016442.png', 'fer0016455.png', 'fer0016461.png', 'fer0016471.png', 'fer0016475.png', 'fer0016484.png', 'fer0016494.png', 'fer0016512.png', 'fer0016514.png', 'fer0016523.png', 'fer0016531.png', 'fer0016536.png', 'fer0016541.png', 'fer0016548.png', 'fer0016555.png', 'fer0016573.png', 'fer0016580.png', 'fer0016592.png', 'fer0016600.png', 'fer0016605.png', 'fer0016610.png', 'fer0016616.png', 'fer0016622.png', 'fer0016641.png', 'fer0016649.png', 'fer0016650.png', 'fer0016661.png', 'fer0016666.png', 'fer0016674.png', 'fer0016683.png', 'fer0016688.png', 'fer0016694.png', 'fer0016696.png', 'fer0016722.png', 'fer0016727.png', 'fer0016732.png', 'fer0016752.png', 'fer0016768.png', 'fer0016772.png', 'fer0016788.png', 'fer0016806.png', 'fer0016825.png', 'fer0016828.png', 'fer0016831.png', 'fer0016833.png', 'fer0016835.png', 'fer0016860.png', 'fer0016869.png', 'fer0016870.png', 'fer0016871.png', 'fer0016888.png', 'fer0016899.png', 'fer0016903.png', 'fer0016920.png', 'fer0016928.png', 'fer0016939.png', 'fer0016944.png', 'fer0016964.png', 'fer0016970.png', 'fer0016973.png', 'fer0016987.png', 'fer0017008.png', 'fer0017028.png', 'fer0017034.png', 'fer0017042.png', 'fer0017049.png', 'fer0017052.png', 'fer0017055.png', 'fer0017061.png', 'fer0017064.png', 'fer0017068.png', 'fer0017070.png', 'fer0017073.png', 'fer0017089.png', 'fer0017096.png', 'fer0017106.png', 'fer0017107.png', 'fer0017110.png', 'fer0017113.png', 'fer0017127.png', 'fer0017135.png', 'fer0017145.png', 'fer0017152.png', 'fer0017179.png', 'fer0017181.png', 'fer0017198.png', 'fer0017199.png', 'fer0017206.png', 'fer0017208.png', 'fer0017218.png', 'fer0017232.png', 'fer0017235.png', 'fer0017247.png', 'fer0017248.png', 'fer0017254.png', 'fer0017256.png', 'fer0017261.png', 'fer0017270.png', 'fer0017281.png', 'fer0017330.png', 'fer0017337.png', 'fer0017343.png', 'fer0017348.png', 'fer0017365.png', 'fer0017374.png', 'fer0017381.png', 'fer0017386.png', 'fer0017394.png', 'fer0017395.png', 'fer0017396.png', 'fer0017397.png', 'fer0017399.png', 'fer0017402.png', 'fer0017407.png', 'fer0017411.png', 'fer0017414.png', 'fer0017455.png', 'fer0017474.png', 'fer0017475.png', 'fer0017486.png', 'fer0017496.png', 'fer0017501.png', 'fer0017514.png', 'fer0017518.png', 'fer0017521.png', 'fer0017523.png', 'fer0017525.png', 'fer0017537.png', 'fer0017538.png', 'fer0017540.png', 'fer0017549.png', 'fer0017572.png', 'fer0017574.png', 'fer0017589.png', 'fer0017590.png', 'fer0017592.png', 'fer0017595.png', 'fer0017597.png', 'fer0017602.png', 'fer0017605.png', 'fer0017617.png', 'fer0017620.png', 'fer0017623.png', 'fer0017633.png', 'fer0017651.png', 'fer0017660.png', 'fer0017661.png', 'fer0017666.png', 'fer0017668.png', 'fer0017692.png', 'fer0017700.png', 'fer0017719.png', 'fer0017726.png', 'fer0017728.png', 'fer0017739.png', 'fer0017741.png', 'fer0017746.png', 'fer0017752.png', 'fer0017764.png', 'fer0017768.png', 'fer0017769.png', 'fer0017770.png', 'fer0017772.png', 'fer0017775.png', 'fer0017781.png', 'fer0017805.png', 'fer0017807.png', 'fer0017808.png', 'fer0017811.png', 'fer0017815.png', 'fer0017832.png', 'fer0017839.png', 'fer0017853.png', 'fer0017865.png', 'fer0017872.png', 'fer0017884.png', 'fer0017891.png', 'fer0017897.png', 'fer0017907.png', 'fer0017908.png', 'fer0017935.png', 'fer0017951.png', 'fer0017952.png', 'fer0017958.png', 'fer0017960.png', 'fer0017961.png', 'fer0017962.png', 'fer0017982.png', 'fer0017992.png', 'fer0017993.png', 'fer0018002.png', 'fer0018009.png', 'fer0018012.png', 'fer0018016.png', 'fer0018030.png', 'fer0018033.png', 'fer0018051.png', 'fer0018053.png', 'fer0018058.png', 'fer0018059.png', 'fer0018064.png', 'fer0018072.png', 'fer0018086.png', 'fer0018088.png', 'fer0018094.png', 'fer0018103.png', 'fer0018128.png', 'fer0018137.png', 'fer0018173.png', 'fer0018174.png', 'fer0018194.png', 'fer0018195.png', 'fer0018212.png', 'fer0018218.png', 'fer0018221.png', 'fer0018227.png', 'fer0018237.png', 'fer0018240.png', 'fer0018244.png', 'fer0018256.png', 'fer0018263.png', 'fer0018270.png', 'fer0018271.png', 'fer0018284.png', 'fer0018300.png', 'fer0018304.png', 'fer0018325.png', 'fer0018328.png', 'fer0018331.png', 'fer0018336.png', 'fer0018361.png', 'fer0018368.png', 'fer0018372.png', 'fer0018405.png', 'fer0018409.png', 'fer0018419.png', 'fer0018425.png', 'fer0018445.png', 'fer0018464.png', 'fer0018469.png', 'fer0018472.png', 'fer0018473.png', 'fer0018480.png', 'fer0018484.png', 'fer0018497.png', 'fer0018500.png', 'fer0018502.png', 'fer0018512.png', 'fer0018521.png', 'fer0018536.png', 'fer0018538.png', 'fer0018543.png', 'fer0018550.png', 'fer0018559.png', 'fer0018563.png', 'fer0018564.png', 'fer0018587.png', 'fer0018588.png', 'fer0018603.png', 'fer0018606.png', 'fer0018622.png', 'fer0018628.png', 'fer0018633.png', 'fer0018640.png', 'fer0018645.png', 'fer0018647.png', 'fer0018651.png', 'fer0018691.png', 'fer0018701.png', 'fer0018704.png', 'fer0018708.png', 'fer0018711.png', 'fer0018713.png', 'fer0018716.png', 'fer0018723.png', 'fer0018726.png', 'fer0018735.png', 'fer0018740.png', 'fer0018741.png', 'fer0018748.png', 'fer0018759.png', 'fer0018764.png', 'fer0018773.png', 'fer0018774.png', 'fer0018780.png', 'fer0018789.png', 'fer0018803.png', 'fer0018809.png', 'fer0018810.png', 'fer0018820.png', 'fer0018823.png', 'fer0018833.png', 'fer0018838.png', 'fer0018842.png', 'fer0018845.png', 'fer0018850.png', 'fer0018858.png', 'fer0018861.png', 'fer0018867.png', 'fer0018869.png', 'fer0018877.png', 'fer0018883.png', 'fer0018892.png', 'fer0018897.png', 'fer0018905.png', 'fer0018923.png', 'fer0018933.png', 'fer0018934.png', 'fer0018935.png', 'fer0018936.png', 'fer0018946.png', 'fer0018948.png', 'fer0018952.png', 'fer0018953.png', 'fer0018956.png', 'fer0018957.png', 'fer0018968.png', 'fer0018973.png', 'fer0019002.png', 'fer0019009.png', 'fer0019010.png', 'fer0019012.png', 'fer0019019.png', 'fer0019023.png', 'fer0019032.png', 'fer0019040.png', 'fer0019042.png', 'fer0019052.png', 'fer0019054.png', 'fer0019057.png', 'fer0019059.png', 'fer0019061.png', 'fer0019086.png', 'fer0019104.png', 'fer0019109.png', 'fer0019121.png', 'fer0019122.png', 'fer0019133.png', 'fer0019138.png', 'fer0019148.png', 'fer0019153.png', 'fer0019183.png', 'fer0019185.png', 'fer0019216.png', 'fer0019245.png', 'fer0019259.png', 'fer0019260.png', 'fer0019272.png', 'fer0019288.png', 'fer0019294.png', 'fer0019299.png', 'fer0019302.png', 'fer0019306.png', 'fer0019319.png', 'fer0019322.png', 'fer0019324.png', 'fer0019332.png', 'fer0019333.png', 'fer0019338.png', 'fer0019341.png', 'fer0019342.png', 'fer0019346.png', 'fer0019353.png', 'fer0019354.png', 'fer0019360.png', 'fer0019370.png', 'fer0019396.png', 'fer0019399.png', 'fer0019401.png', 'fer0019408.png', 'fer0019423.png', 'fer0019434.png', 'fer0019435.png', 'fer0019438.png', 'fer0019441.png', 'fer0019452.png', 'fer0019453.png', 'fer0019454.png', 'fer0019460.png', 'fer0019468.png', 'fer0019473.png', 'fer0019475.png', 'fer0019494.png', 'fer0019502.png', 'fer0019507.png', 'fer0019515.png', 'fer0019519.png', 'fer0019531.png', 'fer0019536.png', 'fer0019549.png', 'fer0019550.png', 'fer0019561.png', 'fer0019562.png', 'fer0019569.png', 'fer0019582.png', 'fer0019588.png', 'fer0019589.png', 'fer0019599.png', 'fer0019600.png', 'fer0019611.png', 'fer0019632.png', 'fer0019639.png', 'fer0019651.png', 'fer0019670.png', 'fer0019681.png', 'fer0019689.png', 'fer0019702.png', 'fer0019703.png', 'fer0019718.png', 'fer0019728.png', 'fer0019729.png', 'fer0019734.png', 'fer0019749.png', 'fer0019751.png', 'fer0019759.png', 'fer0019775.png', 'fer0019782.png', 'fer0019787.png', 'fer0019789.png', 'fer0019791.png', 'fer0019822.png', 'fer0019825.png', 'fer0019840.png', 'fer0019846.png', 'fer0019849.png', 'fer0019852.png', 'fer0019887.png', 'fer0019903.png', 'fer0019911.png', 'fer0019922.png', 'fer0019926.png', 'fer0019937.png', 'fer0019948.png', 'fer0019960.png', 'fer0019965.png', 'fer0019970.png', 'fer0019971.png', 'fer0019980.png', 'fer0019983.png', 'fer0019986.png', 'fer0019989.png', 'fer0019995.png', 'fer0019999.png', 'fer0020001.png', 'fer0020016.png', 'fer0020022.png', 'fer0020027.png', 'fer0020030.png', 'fer0020042.png', 'fer0020052.png', 'fer0020105.png', 'fer0020117.png', 'fer0020131.png', 'fer0020136.png', 'fer0020161.png', 'fer0020169.png', 'fer0020179.png', 'fer0020180.png', 'fer0020196.png', 'fer0020211.png', 'fer0020212.png', 'fer0020213.png', 'fer0020242.png', 'fer0020244.png', 'fer0020250.png', 'fer0020253.png', 'fer0020260.png', 'fer0020269.png', 'fer0020275.png', 'fer0020277.png', 'fer0020290.png', 'fer0020296.png', 'fer0020297.png', 'fer0020306.png', 'fer0020310.png', 'fer0020312.png', 'fer0020319.png', 'fer0020320.png', 'fer0020326.png', 'fer0020333.png', 'fer0020347.png', 'fer0020349.png', 'fer0020350.png', 'fer0020375.png', 'fer0020377.png', 'fer0020379.png', 'fer0020381.png', 'fer0020388.png', 'fer0020401.png', 'fer0020403.png', 'fer0020408.png', 'fer0020422.png', 'fer0020428.png', 'fer0020430.png', 'fer0020434.png', 'fer0020437.png', 'fer0020438.png', 'fer0020440.png', 'fer0020479.png', 'fer0020480.png', 'fer0020492.png', 'fer0020499.png', 'fer0020507.png', 'fer0020518.png', 'fer0020532.png', 'fer0020534.png', 'fer0020538.png', 'fer0020549.png', 'fer0020557.png', 'fer0020579.png', 'fer0020584.png', 'fer0020590.png', 'fer0020600.png', 'fer0020601.png', 'fer0020602.png', 'fer0020606.png', 'fer0020635.png', 'fer0020642.png', 'fer0020648.png', 'fer0020652.png', 'fer0020665.png', 'fer0020668.png', 'fer0020671.png', 'fer0020677.png', 'fer0020686.png', 'fer0020689.png', 'fer0020695.png', 'fer0020697.png', 'fer0020719.png', 'fer0020727.png', 'fer0020728.png', 'fer0020742.png', 'fer0020764.png', 'fer0020774.png', 'fer0020781.png', 'fer0020784.png', 'fer0020805.png', 'fer0020810.png', 'fer0020816.png', 'fer0020822.png', 'fer0020825.png', 'fer0020827.png', 'fer0020834.png', 'fer0020838.png', 'fer0020843.png', 'fer0020846.png', 'fer0020863.png', 'fer0020870.png', 'fer0020872.png', 'fer0020875.png', 'fer0020876.png', 'fer0020878.png', 'fer0020904.png', 'fer0020905.png', 'fer0020906.png', 'fer0020918.png', 'fer0020943.png', 'fer0020944.png', 'fer0020946.png', 'fer0020990.png', 'fer0020995.png', 'fer0021007.png', 'fer0021014.png', 'fer0021015.png', 'fer0021021.png', 'fer0021030.png', 'fer0021037.png', 'fer0021056.png', 'fer0021074.png', 'fer0021077.png', 'fer0021079.png', 'fer0021088.png', 'fer0021094.png', 'fer0021105.png', 'fer0021107.png', 'fer0021120.png', 'fer0021126.png', 'fer0021142.png', 'fer0021155.png', 'fer0021160.png', 'fer0021167.png', 'fer0021168.png', 'fer0021172.png', 'fer0021173.png', 'fer0021177.png', 'fer0021198.png', 'fer0021206.png', 'fer0021207.png', 'fer0021212.png', 'fer0021221.png', 'fer0021233.png', 'fer0021242.png', 'fer0021250.png', 'fer0021255.png', 'fer0021267.png', 'fer0021273.png', 'fer0021276.png', 'fer0021281.png', 'fer0021285.png', 'fer0021291.png', 'fer0021293.png', 'fer0021310.png', 'fer0021311.png', 'fer0021318.png', 'fer0021325.png', 'fer0021339.png', 'fer0021347.png', 'fer0021365.png', 'fer0021374.png', 'fer0021380.png', 'fer0021399.png', 'fer0021405.png', 'fer0021407.png', 'fer0021408.png', 'fer0021409.png', 'fer0021413.png', 'fer0021424.png', 'fer0021432.png', 'fer0021445.png', 'fer0021450.png', 'fer0021455.png', 'fer0021460.png', 'fer0021461.png', 'fer0021478.png', 'fer0021494.png', 'fer0021499.png', 'fer0021501.png', 'fer0021521.png', 'fer0021522.png', 'fer0021523.png', 'fer0021557.png', 'fer0021565.png', 'fer0021567.png', 'fer0021572.png', 'fer0021581.png', 'fer0021593.png', 'fer0021595.png', 'fer0021597.png', 'fer0021607.png', 'fer0021617.png', 'fer0021624.png', 'fer0021626.png', 'fer0021627.png', 'fer0021632.png', 'fer0021635.png', 'fer0021649.png', 'fer0021658.png', 'fer0021674.png', 'fer0021679.png', 'fer0021691.png', 'fer0021694.png', 'fer0021706.png', 'fer0021716.png', 'fer0021723.png', 'fer0021724.png', 'fer0021728.png', 'fer0021732.png', 'fer0021752.png', 'fer0021759.png', 'fer0021786.png', 'fer0021801.png', 'fer0021804.png', 'fer0021805.png', 'fer0021851.png', 'fer0021858.png', 'fer0021862.png', 'fer0021874.png', 'fer0021876.png', 'fer0021877.png', 'fer0021921.png', 'fer0021922.png', 'fer0021932.png', 'fer0021947.png', 'fer0021951.png', 'fer0021953.png', 'fer0021956.png', 'fer0021957.png', 'fer0021958.png', 'fer0021960.png', 'fer0021963.png', 'fer0021965.png', 'fer0021969.png', 'fer0021988.png', 'fer0021999.png', 'fer0022013.png', 'fer0022021.png', 'fer0022023.png', 'fer0022026.png', 'fer0022036.png', 'fer0022040.png', 'fer0022049.png', 'fer0022070.png', 'fer0022085.png', 'fer0022087.png', 'fer0022088.png', 'fer0022092.png', 'fer0022093.png', 'fer0022097.png', 'fer0022100.png', 'fer0022101.png', 'fer0022110.png', 'fer0022117.png', 'fer0022118.png', 'fer0022119.png', 'fer0022124.png', 'fer0022126.png', 'fer0022130.png', 'fer0022138.png', 'fer0022146.png', 'fer0022153.png', 'fer0022164.png', 'fer0022170.png', 'fer0022179.png', 'fer0022180.png', 'fer0022183.png', 'fer0022184.png', 'fer0022191.png', 'fer0022192.png', 'fer0022200.png', 'fer0022211.png', 'fer0022212.png', 'fer0022230.png', 'fer0022239.png', 'fer0022248.png', 'fer0022255.png', 'fer0022256.png', 'fer0022261.png', 'fer0022265.png', 'fer0022269.png', 'fer0022279.png', 'fer0022283.png', 'fer0022290.png', 'fer0022293.png', 'fer0022304.png', 'fer0022312.png', 'fer0022317.png', 'fer0022346.png', 'fer0022355.png', 'fer0022376.png', 'fer0022377.png', 'fer0022379.png', 'fer0022387.png', 'fer0022388.png', 'fer0022400.png', 'fer0022402.png', 'fer0022410.png', 'fer0022415.png', 'fer0022419.png', 'fer0022428.png', 'fer0022429.png', 'fer0022437.png', 'fer0022451.png', 'fer0022458.png', 'fer0022463.png', 'fer0022493.png', 'fer0022494.png', 'fer0022504.png', 'fer0022505.png', 'fer0022526.png', 'fer0022531.png', 'fer0022534.png', 'fer0022538.png', 'fer0022544.png', 'fer0022547.png', 'fer0022568.png', 'fer0022579.png', 'fer0022603.png', 'fer0022610.png', 'fer0022613.png', 'fer0022627.png', 'fer0022639.png', 'fer0022671.png', 'fer0022673.png', 'fer0022675.png', 'fer0022681.png', 'fer0022687.png', 'fer0022688.png', 'fer0022694.png', 'fer0022696.png', 'fer0022702.png', 'fer0022709.png', 'fer0022712.png', 'fer0022719.png', 'fer0022737.png', 'fer0022763.png', 'fer0022770.png', 'fer0022774.png', 'fer0022779.png', 'fer0022802.png', 'fer0022807.png', 'fer0022826.png', 'fer0022883.png', 'fer0022889.png', 'fer0022929.png', 'fer0022930.png', 'fer0022932.png', 'fer0022934.png', 'fer0022937.png', 'fer0022957.png', 'fer0022969.png', 'fer0023001.png', 'fer0023013.png', 'fer0023014.png', 'fer0023020.png', 'fer0023051.png', 'fer0023060.png', 'fer0023066.png', 'fer0023068.png', 'fer0023070.png', 'fer0023074.png', 'fer0023078.png', 'fer0023086.png', 'fer0023087.png', 'fer0023090.png', 'fer0023095.png', 'fer0023107.png', 'fer0023109.png', 'fer0023118.png', 'fer0023133.png', 'fer0023146.png', 'fer0023148.png', 'fer0023152.png', 'fer0023158.png', 'fer0023159.png', 'fer0023164.png', 'fer0023167.png', 'fer0023168.png', 'fer0023183.png', 'fer0023194.png', 'fer0023200.png', 'fer0023202.png', 'fer0023213.png', 'fer0023227.png', 'fer0023228.png', 'fer0023239.png', 'fer0023241.png', 'fer0023246.png', 'fer0023250.png', 'fer0023253.png', 'fer0023256.png', 'fer0023258.png', 'fer0023264.png', 'fer0023295.png', 'fer0023316.png', 'fer0023323.png', 'fer0023331.png', 'fer0023332.png', 'fer0023340.png', 'fer0023343.png', 'fer0023391.png', 'fer0023393.png', 'fer0023396.png', 'fer0023407.png', 'fer0023409.png', 'fer0023415.png', 'fer0023416.png', 'fer0023423.png', 'fer0023425.png', 'fer0023427.png', 'fer0023445.png', 'fer0023446.png', 'fer0023447.png', 'fer0023448.png', 'fer0023450.png', 'fer0023460.png', 'fer0023465.png', 'fer0023469.png', 'fer0023483.png', 'fer0023487.png', 'fer0023503.png', 'fer0023510.png', 'fer0023554.png', 'fer0023556.png', 'fer0023563.png', 'fer0023568.png', 'fer0023571.png', 'fer0023582.png', 'fer0023597.png', 'fer0023605.png', 'fer0023615.png', 'fer0023618.png', 'fer0023621.png', 'fer0023635.png', 'fer0023640.png', 'fer0023641.png', 'fer0023654.png', 'fer0023688.png', 'fer0023691.png', 'fer0023694.png', 'fer0023715.png', 'fer0023723.png', 'fer0023724.png', 'fer0023754.png', 'fer0023757.png', 'fer0023770.png', 'fer0023774.png', 'fer0023780.png', 'fer0023784.png', 'fer0023793.png', 'fer0023794.png', 'fer0023796.png', 'fer0023808.png', 'fer0023811.png', 'fer0023816.png', 'fer0023823.png', 'fer0023832.png', 'fer0023839.png', 'fer0023840.png', 'fer0023844.png', 'fer0023849.png', 'fer0023850.png', 'fer0023870.png', 'fer0023873.png', 'fer0023889.png', 'fer0023902.png', 'fer0023909.png', 'fer0023916.png', 'fer0023919.png', 'fer0023920.png', 'fer0023921.png', 'fer0023927.png', 'fer0023930.png', 'fer0023932.png', 'fer0023933.png', 'fer0023938.png', 'fer0023943.png', 'fer0023945.png', 'fer0023947.png', 'fer0023956.png', 'fer0023966.png', 'fer0023967.png', 'fer0023971.png', 'fer0023972.png', 'fer0023986.png', 'fer0024005.png', 'fer0024015.png', 'fer0024016.png', 'fer0024028.png', 'fer0024051.png', 'fer0024059.png', 'fer0024064.png', 'fer0024068.png', 'fer0024082.png', 'fer0024083.png', 'fer0024102.png', 'fer0024106.png', 'fer0024119.png', 'fer0024122.png', 'fer0024130.png', 'fer0024133.png', 'fer0024146.png', 'fer0024159.png', 'fer0024160.png', 'fer0024163.png', 'fer0024171.png', 'fer0024188.png', 'fer0024200.png', 'fer0024214.png', 'fer0024215.png', 'fer0024228.png', 'fer0024242.png', 'fer0024249.png', 'fer0024256.png', 'fer0024257.png', 'fer0024264.png', 'fer0024270.png', 'fer0024304.png', 'fer0024306.png', 'fer0024308.png', 'fer0024331.png', 'fer0024338.png', 'fer0024339.png', 'fer0024359.png', 'fer0024365.png', 'fer0024368.png', 'fer0024371.png', 'fer0024389.png', 'fer0024396.png', 'fer0024400.png', 'fer0024408.png', 'fer0024417.png', 'fer0024424.png', 'fer0024429.png', 'fer0024452.png', 'fer0024457.png', 'fer0024465.png', 'fer0024478.png', 'fer0024487.png', 'fer0024508.png', 'fer0024513.png', 'fer0024517.png', 'fer0024523.png', 'fer0024538.png', 'fer0024552.png', 'fer0024573.png', 'fer0024580.png', 'fer0024592.png', 'fer0024600.png', 'fer0024605.png', 'fer0024608.png', 'fer0024620.png', 'fer0024624.png', 'fer0024631.png', 'fer0024642.png', 'fer0024647.png', 'fer0024651.png', 'fer0024652.png', 'fer0024654.png', 'fer0024659.png', 'fer0024671.png', 'fer0024673.png', 'fer0024676.png', 'fer0024679.png', 'fer0024695.png', 'fer0024697.png', 'fer0024702.png', 'fer0024703.png', 'fer0024704.png', 'fer0024710.png', 'fer0024712.png', 'fer0024713.png', 'fer0024727.png', 'fer0024733.png', 'fer0024734.png', 'fer0024745.png', 'fer0024754.png', 'fer0024755.png', 'fer0024768.png', 'fer0024785.png', 'fer0024804.png', 'fer0024823.png', 'fer0024824.png', 'fer0024827.png', 'fer0024831.png', 'fer0024832.png', 'fer0024839.png', 'fer0024841.png', 'fer0024852.png', 'fer0024903.png', 'fer0024915.png', 'fer0024931.png', 'fer0024939.png', 'fer0024951.png', 'fer0024954.png', 'fer0024964.png', 'fer0024971.png', 'fer0024978.png', 'fer0024981.png', 'fer0024984.png', 'fer0024986.png', 'fer0024987.png', 'fer0024989.png', 'fer0024996.png', 'fer0025002.png', 'fer0025008.png', 'fer0025011.png', 'fer0025020.png', 'fer0025024.png', 'fer0025028.png', 'fer0025050.png', 'fer0025063.png', 'fer0025069.png', 'fer0025076.png', 'fer0025088.png', 'fer0025094.png', 'fer0025110.png', 'fer0025118.png', 'fer0025120.png', 'fer0025130.png', 'fer0025138.png', 'fer0025156.png', 'fer0025164.png', 'fer0025178.png', 'fer0025185.png', 'fer0025201.png', 'fer0025203.png', 'fer0025207.png', 'fer0025211.png', 'fer0025227.png', 'fer0025229.png', 'fer0025237.png', 'fer0025280.png', 'fer0025284.png', 'fer0025286.png', 'fer0025293.png', 'fer0025299.png', 'fer0025304.png', 'fer0025309.png', 'fer0025310.png', 'fer0025314.png', 'fer0025325.png', 'fer0025333.png', 'fer0025341.png', 'fer0025347.png', 'fer0025356.png', 'fer0025360.png', 'fer0025379.png', 'fer0025397.png', 'fer0025398.png', 'fer0025421.png', 'fer0025422.png', 'fer0025429.png', 'fer0025463.png', 'fer0025464.png', 'fer0025475.png', 'fer0025485.png', 'fer0025497.png', 'fer0025501.png', 'fer0025512.png', 'fer0025535.png', 'fer0025556.png', 'fer0025576.png', 'fer0025578.png', 'fer0025582.png', 'fer0025595.png', 'fer0025599.png', 'fer0025604.png', 'fer0025618.png', 'fer0025620.png', 'fer0025635.png', 'fer0025643.png', 'fer0025647.png', 'fer0025664.png', 'fer0025665.png', 'fer0025667.png', 'fer0025669.png', 'fer0025671.png', 'fer0025674.png', 'fer0025689.png', 'fer0025690.png', 'fer0025696.png', 'fer0025726.png', 'fer0025736.png', 'fer0025747.png', 'fer0025754.png', 'fer0025763.png', 'fer0025774.png', 'fer0025777.png', 'fer0025785.png', 'fer0025788.png', 'fer0025790.png', 'fer0025791.png', 'fer0025793.png', 'fer0025796.png', 'fer0025806.png', 'fer0025807.png', 'fer0025808.png', 'fer0025810.png', 'fer0025811.png', 'fer0025814.png', 'fer0025829.png', 'fer0025834.png', 'fer0025838.png', 'fer0025843.png', 'fer0025857.png', 'fer0025862.png', 'fer0025873.png', 'fer0025874.png', 'fer0025878.png', 'fer0025888.png', 'fer0025892.png', 'fer0025912.png', 'fer0025918.png', 'fer0025928.png', 'fer0025930.png', 'fer0025935.png', 'fer0025952.png', 'fer0025953.png', 'fer0025954.png', 'fer0025961.png', 'fer0025967.png', 'fer0025968.png', 'fer0025978.png', 'fer0025979.png', 'fer0025985.png', 'fer0025993.png', 'fer0026017.png', 'fer0026020.png', 'fer0026022.png', 'fer0026026.png', 'fer0026033.png', 'fer0026038.png', 'fer0026043.png', 'fer0026056.png', 'fer0026058.png', 'fer0026061.png', 'fer0026069.png', 'fer0026072.png', 'fer0026083.png', 'fer0026085.png', 'fer0026089.png', 'fer0026096.png', 'fer0026103.png', 'fer0026105.png', 'fer0026107.png', 'fer0026111.png', 'fer0026113.png', 'fer0026124.png', 'fer0026157.png', 'fer0026174.png', 'fer0026181.png', 'fer0026188.png', 'fer0026192.png', 'fer0026199.png', 'fer0026200.png', 'fer0026219.png', 'fer0026225.png', 'fer0026227.png', 'fer0026229.png', 'fer0026231.png', 'fer0026245.png', 'fer0026262.png', 'fer0026266.png', 'fer0026291.png', 'fer0026299.png', 'fer0026314.png', 'fer0026340.png', 'fer0026345.png', 'fer0026350.png', 'fer0026355.png', 'fer0026360.png', 'fer0026361.png', 'fer0026365.png', 'fer0026370.png', 'fer0026377.png', 'fer0026386.png', 'fer0026412.png', 'fer0026414.png', 'fer0026423.png', 'fer0026433.png', 'fer0026446.png', 'fer0026465.png', 'fer0026477.png', 'fer0026481.png', 'fer0026492.png', 'fer0026498.png', 'fer0026506.png', 'fer0026510.png', 'fer0026512.png', 'fer0026517.png', 'fer0026542.png', 'fer0026547.png', 'fer0026548.png', 'fer0026560.png', 'fer0026565.png', 'fer0026593.png', 'fer0026611.png', 'fer0026613.png', 'fer0026615.png', 'fer0026629.png', 'fer0026633.png', 'fer0026636.png', 'fer0026649.png', 'fer0026658.png', 'fer0026661.png', 'fer0026664.png', 'fer0026671.png', 'fer0026672.png', 'fer0026673.png', 'fer0026676.png', 'fer0026679.png', 'fer0026693.png', 'fer0026697.png', 'fer0026715.png', 'fer0026721.png', 'fer0026725.png', 'fer0026746.png', 'fer0026773.png', 'fer0026777.png', 'fer0026783.png', 'fer0026792.png', 'fer0026800.png', 'fer0026804.png', 'fer0026829.png', 'fer0026830.png', 'fer0026838.png', 'fer0026841.png', 'fer0026844.png', 'fer0026855.png', 'fer0026857.png', 'fer0026880.png', 'fer0026899.png', 'fer0026909.png', 'fer0026910.png', 'fer0026923.png', 'fer0026925.png', 'fer0026928.png', 'fer0026930.png', 'fer0026951.png', 'fer0026958.png', 'fer0026960.png', 'fer0026970.png', 'fer0026973.png', 'fer0026974.png', 'fer0026975.png', 'fer0026982.png', 'fer0026987.png', 'fer0026998.png', 'fer0027000.png', 'fer0027017.png', 'fer0027019.png', 'fer0027027.png', 'fer0027039.png', 'fer0027043.png', 'fer0027048.png', 'fer0027049.png', 'fer0027074.png', 'fer0027075.png', 'fer0027095.png', 'fer0027119.png', 'fer0027130.png', 'fer0027139.png', 'fer0027143.png', 'fer0027144.png', 'fer0027145.png', 'fer0027147.png', 'fer0027158.png', 'fer0027163.png', 'fer0027164.png', 'fer0027168.png', 'fer0027174.png', 'fer0027180.png', 'fer0027182.png', 'fer0027191.png', 'fer0027201.png', 'fer0027217.png', 'fer0027226.png', 'fer0027232.png', 'fer0027234.png', 'fer0027246.png', 'fer0027253.png', 'fer0027255.png', 'fer0027259.png', 'fer0027263.png', 'fer0027280.png', 'fer0027286.png', 'fer0027288.png', 'fer0027290.png', 'fer0027292.png', 'fer0027293.png', 'fer0027320.png', 'fer0027322.png', 'fer0027327.png', 'fer0027331.png', 'fer0027338.png', 'fer0027339.png', 'fer0027348.png', 'fer0027350.png', 'fer0027352.png', 'fer0027359.png', 'fer0027360.png', 'fer0027362.png', 'fer0027365.png', 'fer0027369.png', 'fer0027371.png', 'fer0027373.png', 'fer0027389.png', 'fer0027390.png', 'fer0027398.png', 'fer0027403.png', 'fer0027429.png', 'fer0027443.png', 'fer0027477.png', 'fer0027491.png', 'fer0027502.png', 'fer0027508.png', 'fer0027513.png', 'fer0027521.png', 'fer0027523.png', 'fer0027527.png', 'fer0027569.png', 'fer0027580.png', 'fer0027592.png', 'fer0027595.png', 'fer0027599.png', 'fer0027600.png', 'fer0027602.png', 'fer0027604.png', 'fer0027628.png', 'fer0027634.png', 'fer0027638.png', 'fer0027647.png', 'fer0027650.png', 'fer0027653.png', 'fer0027685.png', 'fer0027692.png', 'fer0027705.png', 'fer0027720.png', 'fer0027723.png', 'fer0027737.png', 'fer0027771.png', 'fer0027776.png', 'fer0027779.png', 'fer0027787.png', 'fer0027802.png', 'fer0027804.png', 'fer0027810.png', 'fer0027813.png', 'fer0027817.png', 'fer0027824.png', 'fer0027832.png', 'fer0027833.png', 'fer0027845.png', 'fer0027851.png', 'fer0027855.png', 'fer0027859.png', 'fer0027866.png', 'fer0027897.png', 'fer0027911.png', 'fer0027929.png', 'fer0027934.png', 'fer0027936.png', 'fer0027940.png', 'fer0027949.png', 'fer0027962.png', 'fer0027977.png', 'fer0027978.png', 'fer0027988.png', 'fer0027994.png', 'fer0028005.png', 'fer0028011.png', 'fer0028014.png', 'fer0028021.png', 'fer0028038.png', 'fer0028040.png', 'fer0028042.png', 'fer0028043.png', 'fer0028050.png', 'fer0028059.png', 'fer0028067.png', 'fer0028073.png', 'fer0028074.png', 'fer0028076.png', 'fer0028081.png', 'fer0028085.png', 'fer0028096.png', 'fer0028097.png', 'fer0028098.png', 'fer0028107.png', 'fer0028108.png', 'fer0028115.png', 'fer0028118.png', 'fer0028128.png', 'fer0028134.png', 'fer0028141.png', 'fer0028164.png', 'fer0028172.png', 'fer0028190.png', 'fer0028193.png', 'fer0028198.png', 'fer0028218.png', 'fer0028222.png', 'fer0028224.png', 'fer0028230.png', 'fer0028260.png', 'fer0028261.png', 'fer0028265.png', 'fer0028277.png', 'fer0028279.png', 'fer0028282.png', 'fer0028319.png', 'fer0028321.png', 'fer0028325.png', 'fer0028338.png', 'fer0028339.png', 'fer0028342.png', 'fer0028350.png', 'fer0028366.png', 'fer0028377.png', 'fer0028382.png', 'fer0028395.png', 'fer0028398.png', 'fer0028410.png', 'fer0028436.png', 'fer0028439.png', 'fer0028443.png', 'fer0028456.png', 'fer0028459.png', 'fer0028464.png', 'fer0028465.png', 'fer0028471.png', 'fer0028472.png', 'fer0028481.png', 'fer0028485.png', 'fer0028488.png', 'fer0028495.png', 'fer0028519.png', 'fer0028528.png', 'fer0028539.png', 'fer0028558.png', 'fer0028562.png', 'fer0028565.png', 'fer0028572.png', 'fer0028598.png', 'fer0028611.png', 'fer0028614.png', 'fer0028620.png', 'fer0028629.png', 'fer0028634.png', 'fer0028635.png']
test_miss = ['fer0032220.png', 'fer0032233.png', 'fer0032234.png', 'fer0032236.png', 'fer0032238.png', 'fer0032239.png', 'fer0032253.png', 'fer0032256.png', 'fer0032262.png', 'fer0032271.png', 'fer0032279.png', 'fer0032280.png', 'fer0032289.png', 'fer0032292.png', 'fer0032297.png', 'fer0032312.png', 'fer0032315.png', 'fer0032331.png', 'fer0032332.png', 'fer0032345.png', 'fer0032352.png', 'fer0032359.png', 'fer0032383.png', 'fer0032399.png', 'fer0032403.png', 'fer0032411.png', 'fer0032414.png', 'fer0032428.png', 'fer0032430.png', 'fer0032433.png', 'fer0032440.png', 'fer0032447.png', 'fer0032455.png', 'fer0032456.png', 'fer0032457.png', 'fer0032464.png', 'fer0032468.png', 'fer0032470.png', 'fer0032472.png', 'fer0032473.png', 'fer0032484.png', 'fer0032485.png', 'fer0032493.png', 'fer0032511.png', 'fer0032521.png', 'fer0032527.png', 'fer0032532.png', 'fer0032546.png', 'fer0032548.png', 'fer0032555.png', 'fer0032559.png', 'fer0032560.png', 'fer0032571.png', 'fer0032576.png', 'fer0032588.png', 'fer0032594.png', 'fer0032611.png', 'fer0032617.png', 'fer0032629.png', 'fer0032631.png', 'fer0032635.png', 'fer0032656.png', 'fer0032659.png', 'fer0032664.png', 'fer0032666.png', 'fer0032692.png', 'fer0032704.png', 'fer0032705.png', 'fer0032709.png', 'fer0032714.png', 'fer0032728.png', 'fer0032741.png', 'fer0032744.png', 'fer0032749.png', 'fer0032757.png', 'fer0032773.png', 'fer0032775.png', 'fer0032785.png', 'fer0032787.png', 'fer0032797.png', 'fer0032801.png', 'fer0032804.png', 'fer0032826.png', 'fer0032836.png', 'fer0032843.png', 'fer0032860.png', 'fer0032866.png', 'fer0032881.png', 'fer0032882.png', 'fer0032894.png', 'fer0032906.png', 'fer0032910.png', 'fer0032914.png', 'fer0032924.png', 'fer0032939.png', 'fer0032947.png', 'fer0032968.png', 'fer0032977.png', 'fer0032981.png', 'fer0032982.png', 'fer0032986.png', 'fer0032990.png', 'fer0032994.png', 'fer0033003.png', 'fer0033013.png', 'fer0033016.png', 'fer0033025.png', 'fer0033028.png', 'fer0033029.png', 'fer0033037.png', 'fer0033043.png', 'fer0033048.png', 'fer0033052.png', 'fer0033068.png', 'fer0033104.png', 'fer0033111.png', 'fer0033120.png', 'fer0033130.png', 'fer0033135.png', 'fer0033139.png', 'fer0033141.png', 'fer0033153.png', 'fer0033159.png', 'fer0033174.png', 'fer0033180.png', 'fer0033227.png', 'fer0033236.png', 'fer0033241.png', 'fer0033252.png', 'fer0033254.png', 'fer0033270.png', 'fer0033273.png', 'fer0033283.png', 'fer0033289.png', 'fer0033292.png', 'fer0033304.png', 'fer0033306.png', 'fer0033312.png', 'fer0033314.png', 'fer0033336.png', 'fer0033370.png', 'fer0033378.png', 'fer0033395.png', 'fer0033415.png', 'fer0033416.png', 'fer0033419.png', 'fer0033432.png', 'fer0033434.png', 'fer0033435.png', 'fer0033458.png', 'fer0033461.png', 'fer0033462.png', 'fer0033463.png', 'fer0033468.png', 'fer0033470.png', 'fer0033477.png', 'fer0033497.png', 'fer0033498.png', 'fer0033499.png', 'fer0033505.png', 'fer0033510.png', 'fer0033513.png', 'fer0033517.png', 'fer0033518.png', 'fer0033525.png', 'fer0033533.png', 'fer0033535.png', 'fer0033537.png', 'fer0033547.png', 'fer0033548.png', 'fer0033551.png', 'fer0033560.png', 'fer0033565.png', 'fer0033566.png', 'fer0033568.png', 'fer0033578.png', 'fer0033579.png', 'fer0033586.png', 'fer0033589.png', 'fer0033601.png', 'fer0033612.png', 'fer0033613.png', 'fer0033621.png', 'fer0033635.png', 'fer0033642.png', 'fer0033645.png', 'fer0033658.png', 'fer0033669.png', 'fer0033676.png', 'fer0033682.png', 'fer0033684.png', 'fer0033691.png', 'fer0033710.png', 'fer0033716.png', 'fer0033719.png', 'fer0033731.png', 'fer0033740.png', 'fer0033742.png', 'fer0033766.png', 'fer0033772.png', 'fer0033775.png', 'fer0033777.png', 'fer0033784.png', 'fer0033803.png', 'fer0033804.png', 'fer0033814.png', 'fer0033834.png', 'fer0033837.png', 'fer0033855.png', 'fer0033867.png', 'fer0033879.png', 'fer0033895.png', 'fer0033898.png', 'fer0033906.png', 'fer0033908.png', 'fer0033929.png', 'fer0033942.png', 'fer0033950.png', 'fer0033952.png', 'fer0033954.png', 'fer0033955.png', 'fer0033959.png', 'fer0033963.png', 'fer0033971.png', 'fer0033978.png', 'fer0033982.png', 'fer0033989.png', 'fer0033990.png', 'fer0033991.png', 'fer0033993.png', 'fer0033998.png', 'fer0034003.png', 'fer0034005.png', 'fer0034007.png', 'fer0034011.png', 'fer0034012.png', 'fer0034015.png', 'fer0034022.png', 'fer0034028.png', 'fer0034035.png', 'fer0034036.png', 'fer0034046.png', 'fer0034051.png', 'fer0034053.png', 'fer0034055.png', 'fer0034112.png', 'fer0034121.png', 'fer0034134.png', 'fer0034135.png', 'fer0034144.png', 'fer0034147.png', 'fer0034159.png', 'fer0034160.png', 'fer0034166.png', 'fer0034178.png', 'fer0034187.png', 'fer0034188.png', 'fer0034194.png', 'fer0034212.png', 'fer0034220.png', 'fer0034260.png', 'fer0034261.png', 'fer0034264.png', 'fer0034267.png', 'fer0034269.png', 'fer0034299.png', 'fer0034311.png', 'fer0034312.png', 'fer0034313.png', 'fer0034355.png', 'fer0034362.png', 'fer0034377.png', 'fer0034381.png', 'fer0034390.png', 'fer0034394.png', 'fer0034397.png', 'fer0034399.png', 'fer0034405.png', 'fer0034411.png', 'fer0034414.png', 'fer0034415.png', 'fer0034423.png', 'fer0034424.png', 'fer0034431.png', 'fer0034437.png', 'fer0034463.png', 'fer0034466.png', 'fer0034468.png', 'fer0034471.png', 'fer0034495.png', 'fer0034498.png', 'fer0034501.png', 'fer0034514.png', 'fer0034546.png', 'fer0034575.png', 'fer0034578.png', 'fer0034586.png', 'fer0034596.png', 'fer0034598.png', 'fer0034616.png', 'fer0034617.png', 'fer0034636.png', 'fer0034640.png', 'fer0034648.png', 'fer0034668.png', 'fer0034669.png', 'fer0034673.png', 'fer0034680.png', 'fer0034686.png', 'fer0034689.png', 'fer0034703.png', 'fer0034706.png', 'fer0034707.png', 'fer0034709.png', 'fer0034717.png', 'fer0034721.png', 'fer0034739.png', 'fer0034740.png', 'fer0034757.png', 'fer0034764.png', 'fer0034768.png', 'fer0034772.png', 'fer0034808.png', 'fer0034816.png', 'fer0034828.png', 'fer0034863.png', 'fer0034868.png', 'fer0034871.png', 'fer0034890.png', 'fer0034897.png', 'fer0034901.png', 'fer0034910.png', 'fer0034924.png', 'fer0034928.png', 'fer0034931.png', 'fer0034932.png', 'fer0034935.png', 'fer0034938.png', 'fer0034940.png', 'fer0034945.png', 'fer0034947.png', 'fer0034954.png', 'fer0034955.png', 'fer0034959.png', 'fer0034960.png', 'fer0034973.png', 'fer0034979.png', 'fer0034992.png', 'fer0034999.png', 'fer0035000.png', 'fer0035004.png', 'fer0035014.png', 'fer0035017.png', 'fer0035026.png', 'fer0035038.png', 'fer0035039.png', 'fer0035040.png', 'fer0035053.png', 'fer0035067.png', 'fer0035080.png', 'fer0035088.png', 'fer0035091.png', 'fer0035103.png', 'fer0035108.png', 'fer0035110.png', 'fer0035124.png', 'fer0035136.png', 'fer0035141.png', 'fer0035143.png', 'fer0035147.png', 'fer0035166.png', 'fer0035170.png', 'fer0035178.png', 'fer0035179.png', 'fer0035189.png', 'fer0035200.png', 'fer0035203.png', 'fer0035216.png', 'fer0035220.png', 'fer0035225.png', 'fer0035228.png', 'fer0035230.png', 'fer0035241.png', 'fer0035243.png', 'fer0035244.png', 'fer0035249.png', 'fer0035263.png', 'fer0035270.png', 'fer0035275.png', 'fer0035280.png', 'fer0035319.png', 'fer0035320.png', 'fer0035321.png', 'fer0035331.png', 'fer0035333.png', 'fer0035337.png', 'fer0035345.png', 'fer0035352.png', 'fer0035357.png', 'fer0035384.png', 'fer0035403.png', 'fer0035407.png', 'fer0035408.png', 'fer0035423.png', 'fer0035426.png', 'fer0035429.png', 'fer0035430.png', 'fer0035432.png', 'fer0035442.png', 'fer0035449.png', 'fer0035450.png', 'fer0035458.png', 'fer0035468.png', 'fer0035469.png', 'fer0035471.png', 'fer0035507.png', 'fer0035508.png', 'fer0035514.png', 'fer0035521.png', 'fer0035532.png', 'fer0035541.png', 'fer0035545.png', 'fer0035549.png', 'fer0035572.png', 'fer0035583.png', 'fer0035588.png', 'fer0035598.png', 'fer0035601.png', 'fer0035604.png', 'fer0035605.png', 'fer0035609.png', 'fer0035622.png', 'fer0035639.png', 'fer0035645.png', 'fer0035662.png', 'fer0035666.png', 'fer0035676.png', 'fer0035678.png', 'fer0035681.png', 'fer0035682.png', 'fer0035683.png', 'fer0035685.png', 'fer0035700.png', 'fer0035710.png', 'fer0035730.png', 'fer0035732.png', 'fer0035733.png', 'fer0035739.png', 'fer0035743.png', 'fer0035750.png', 'fer0035753.png', 'fer0035756.png', 'fer0035762.png', 'fer0035764.png', 'fer0035771.png', 'fer0035784.png']
valid_miss = ['fer0028640.png', 'fer0028648.png', 'fer0028653.png', 'fer0028664.png', 'fer0028683.png', 'fer0028694.png', 'fer0028702.png', 'fer0028704.png', 'fer0028706.png', 'fer0028707.png', 'fer0028709.png', 'fer0028718.png', 'fer0028719.png', 'fer0028723.png', 'fer0028732.png', 'fer0028733.png', 'fer0028746.png', 'fer0028749.png', 'fer0028751.png', 'fer0028768.png', 'fer0028781.png', 'fer0028814.png', 'fer0028815.png', 'fer0028816.png', 'fer0028828.png', 'fer0028829.png', 'fer0028845.png', 'fer0028849.png', 'fer0028859.png', 'fer0028862.png', 'fer0028866.png', 'fer0028868.png', 'fer0028873.png', 'fer0028874.png', 'fer0028876.png', 'fer0028895.png', 'fer0028900.png', 'fer0028903.png', 'fer0028908.png', 'fer0028922.png', 'fer0028924.png', 'fer0028934.png', 'fer0028937.png', 'fer0028938.png', 'fer0029007.png', 'fer0029010.png', 'fer0029012.png', 'fer0029021.png', 'fer0029027.png', 'fer0029044.png', 'fer0029060.png', 'fer0029074.png', 'fer0029075.png', 'fer0029079.png', 'fer0029083.png', 'fer0029090.png', 'fer0029093.png', 'fer0029096.png', 'fer0029104.png', 'fer0029113.png', 'fer0029114.png', 'fer0029122.png', 'fer0029132.png', 'fer0029133.png', 'fer0029134.png', 'fer0029155.png', 'fer0029156.png', 'fer0029157.png', 'fer0029166.png', 'fer0029167.png', 'fer0029182.png', 'fer0029184.png', 'fer0029195.png', 'fer0029197.png', 'fer0029200.png', 'fer0029206.png', 'fer0029214.png', 'fer0029224.png', 'fer0029230.png', 'fer0029234.png', 'fer0029236.png', 'fer0029248.png', 'fer0029249.png', 'fer0029270.png', 'fer0029275.png', 'fer0029288.png', 'fer0029295.png', 'fer0029298.png', 'fer0029307.png', 'fer0029330.png', 'fer0029336.png', 'fer0029349.png', 'fer0029368.png', 'fer0029369.png', 'fer0029388.png', 'fer0029409.png', 'fer0029416.png', 'fer0029424.png', 'fer0029449.png', 'fer0029472.png', 'fer0029476.png', 'fer0029488.png', 'fer0029496.png', 'fer0029501.png', 'fer0029513.png', 'fer0029522.png', 'fer0029528.png', 'fer0029537.png', 'fer0029542.png', 'fer0029543.png', 'fer0029544.png', 'fer0029554.png', 'fer0029557.png', 'fer0029565.png', 'fer0029570.png', 'fer0029588.png', 'fer0029590.png', 'fer0029597.png', 'fer0029601.png', 'fer0029612.png', 'fer0029632.png', 'fer0029637.png', 'fer0029639.png', 'fer0029642.png', 'fer0029645.png', 'fer0029648.png', 'fer0029660.png', 'fer0029663.png', 'fer0029664.png', 'fer0029666.png', 'fer0029669.png', 'fer0029673.png', 'fer0029686.png', 'fer0029695.png', 'fer0029707.png', 'fer0029741.png', 'fer0029754.png', 'fer0029757.png', 'fer0029763.png', 'fer0029776.png', 'fer0029779.png', 'fer0029796.png', 'fer0029807.png', 'fer0029810.png', 'fer0029824.png', 'fer0029838.png', 'fer0029841.png', 'fer0029844.png', 'fer0029847.png', 'fer0029849.png', 'fer0029850.png', 'fer0029857.png', 'fer0029864.png', 'fer0029869.png', 'fer0029894.png', 'fer0029895.png', 'fer0029920.png', 'fer0029921.png', 'fer0029932.png', 'fer0029939.png', 'fer0029946.png', 'fer0029947.png', 'fer0029949.png', 'fer0029957.png', 'fer0029958.png', 'fer0029959.png', 'fer0029967.png', 'fer0029982.png', 'fer0030011.png', 'fer0030013.png', 'fer0030014.png', 'fer0030016.png', 'fer0030020.png', 'fer0030038.png', 'fer0030061.png', 'fer0030062.png', 'fer0030075.png', 'fer0030077.png', 'fer0030082.png', 'fer0030097.png', 'fer0030109.png', 'fer0030110.png', 'fer0030112.png', 'fer0030120.png', 'fer0030123.png', 'fer0030145.png', 'fer0030149.png', 'fer0030152.png', 'fer0030153.png', 'fer0030189.png', 'fer0030199.png', 'fer0030205.png', 'fer0030221.png', 'fer0030224.png', 'fer0030225.png', 'fer0030235.png', 'fer0030243.png', 'fer0030252.png', 'fer0030276.png', 'fer0030278.png', 'fer0030287.png', 'fer0030288.png', 'fer0030294.png', 'fer0030295.png', 'fer0030298.png', 'fer0030320.png', 'fer0030328.png', 'fer0030331.png', 'fer0030334.png', 'fer0030335.png', 'fer0030344.png', 'fer0030371.png', 'fer0030374.png', 'fer0030384.png', 'fer0030385.png', 'fer0030386.png', 'fer0030388.png', 'fer0030394.png', 'fer0030403.png', 'fer0030405.png', 'fer0030413.png', 'fer0030421.png', 'fer0030423.png', 'fer0030427.png', 'fer0030431.png', 'fer0030433.png', 'fer0030439.png', 'fer0030445.png', 'fer0030459.png', 'fer0030463.png', 'fer0030464.png', 'fer0030468.png', 'fer0030472.png', 'fer0030478.png', 'fer0030482.png', 'fer0030483.png', 'fer0030495.png', 'fer0030499.png', 'fer0030516.png', 'fer0030518.png', 'fer0030521.png', 'fer0030523.png', 'fer0030532.png', 'fer0030537.png', 'fer0030543.png', 'fer0030547.png', 'fer0030552.png', 'fer0030553.png', 'fer0030560.png', 'fer0030591.png', 'fer0030600.png', 'fer0030606.png', 'fer0030607.png', 'fer0030613.png', 'fer0030618.png', 'fer0030621.png', 'fer0030631.png', 'fer0030637.png', 'fer0030643.png', 'fer0030655.png', 'fer0030671.png', 'fer0030682.png', 'fer0030690.png', 'fer0030708.png', 'fer0030711.png', 'fer0030715.png', 'fer0030717.png', 'fer0030722.png', 'fer0030737.png', 'fer0030755.png', 'fer0030756.png', 'fer0030771.png', 'fer0030784.png', 'fer0030792.png', 'fer0030797.png', 'fer0030802.png', 'fer0030804.png', 'fer0030811.png', 'fer0030814.png', 'fer0030823.png', 'fer0030834.png', 'fer0030836.png', 'fer0030838.png', 'fer0030846.png', 'fer0030861.png', 'fer0030871.png', 'fer0030891.png', 'fer0030894.png', 'fer0030897.png', 'fer0030911.png', 'fer0030912.png', 'fer0030914.png', 'fer0030917.png', 'fer0030920.png', 'fer0030922.png', 'fer0030937.png', 'fer0030938.png', 'fer0030953.png', 'fer0030956.png', 'fer0030960.png', 'fer0030964.png', 'fer0030991.png', 'fer0030992.png', 'fer0031003.png', 'fer0031017.png', 'fer0031020.png', 'fer0031024.png', 'fer0031032.png', 'fer0031039.png', 'fer0031040.png', 'fer0031041.png', 'fer0031046.png', 'fer0031049.png', 'fer0031050.png', 'fer0031055.png', 'fer0031066.png', 'fer0031079.png', 'fer0031083.png', 'fer0031085.png', 'fer0031086.png', 'fer0031093.png', 'fer0031096.png', 'fer0031098.png', 'fer0031104.png', 'fer0031107.png', 'fer0031110.png', 'fer0031114.png', 'fer0031116.png', 'fer0031128.png', 'fer0031130.png', 'fer0031133.png', 'fer0031138.png', 'fer0031152.png', 'fer0031172.png', 'fer0031184.png', 'fer0031187.png', 'fer0031188.png', 'fer0031189.png', 'fer0031192.png', 'fer0031194.png', 'fer0031200.png', 'fer0031207.png', 'fer0031212.png', 'fer0031222.png', 'fer0031250.png', 'fer0031253.png', 'fer0031254.png', 'fer0031256.png', 'fer0031274.png', 'fer0031278.png', 'fer0031291.png', 'fer0031297.png', 'fer0031302.png', 'fer0031309.png', 'fer0031313.png', 'fer0031317.png', 'fer0031323.png', 'fer0031329.png', 'fer0031330.png', 'fer0031335.png', 'fer0031340.png', 'fer0031342.png', 'fer0031350.png', 'fer0031356.png', 'fer0031357.png', 'fer0031365.png', 'fer0031367.png', 'fer0031370.png', 'fer0031372.png', 'fer0031388.png', 'fer0031403.png', 'fer0031411.png', 'fer0031412.png', 'fer0031414.png', 'fer0031420.png', 'fer0031421.png', 'fer0031423.png', 'fer0031426.png', 'fer0031431.png', 'fer0031452.png', 'fer0031459.png', 'fer0031469.png', 'fer0031480.png', 'fer0031485.png', 'fer0031487.png', 'fer0031504.png', 'fer0031512.png', 'fer0031515.png', 'fer0031523.png', 'fer0031527.png', 'fer0031539.png', 'fer0031540.png', 'fer0031545.png', 'fer0031549.png', 'fer0031557.png', 'fer0031559.png', 'fer0031563.png', 'fer0031569.png', 'fer0031591.png', 'fer0031603.png', 'fer0031612.png', 'fer0031619.png', 'fer0031621.png', 'fer0031623.png', 'fer0031628.png', 'fer0031629.png', 'fer0031638.png', 'fer0031641.png', 'fer0031664.png', 'fer0031686.png', 'fer0031691.png', 'fer0031703.png', 'fer0031708.png', 'fer0031711.png', 'fer0031727.png', 'fer0031728.png', 'fer0031731.png', 'fer0031738.png', 'fer0031744.png', 'fer0031759.png', 'fer0031760.png', 'fer0031779.png', 'fer0031788.png', 'fer0031794.png', 'fer0031803.png', 'fer0031809.png', 'fer0031815.png', 'fer0031820.png', 'fer0031844.png', 'fer0031847.png', 'fer0031850.png', 'fer0031857.png', 'fer0031870.png', 'fer0031875.png', 'fer0031876.png', 'fer0031880.png', 'fer0031886.png', 'fer0031914.png', 'fer0031917.png', 'fer0031923.png', 'fer0031940.png', 'fer0031947.png', 'fer0031964.png', 'fer0031970.png', 'fer0031982.png', 'fer0032005.png', 'fer0032008.png', 'fer0032010.png', 'fer0032011.png', 'fer0032014.png', 'fer0032015.png', 'fer0032016.png', 'fer0032033.png', 'fer0032049.png', 'fer0032054.png', 'fer0032063.png', 'fer0032065.png', 'fer0032067.png', 'fer0032074.png', 'fer0032076.png', 'fer0032095.png', 'fer0032106.png', 'fer0032112.png', 'fer0032118.png', 'fer0032128.png', 'fer0032129.png', 'fer0032132.png', 'fer0032133.png', 'fer0032145.png', 'fer0032146.png', 'fer0032156.png', 'fer0032158.png', 'fer0032162.png', 'fer0032163.png', 'fer0032167.png', 'fer0032206.png']

miss_dict = {"FER2013Train": training_miss, "FER2013Test": test_miss, "FER2013Valid": valid_miss}

In [57]:
modes = ["FER2013Train", "FER2013Test", "FER2013Valid"]
columns = ["img_name", "dims", "0", "1", "2", "3", "4", "5", "6", "7", "Unknown", "NF"]

for mode in modes:
    miss_list = miss_dict[mode]
    df = pd.read_csv("../../data/refinedFERPlus/"+mode+"/label.csv", header=None)
    df.columns = columns
    
    print("Before drop miss images in [{}] mode, length of label.csv: {}".format(mode, len(df)))
    
    for miss in miss_list:
        df.drop(index=df.loc[df['img_name'] == miss].index, inplace=True)
    
    print("After drop miss images, length of label.csv: {}".format(len(df)))
    print(df)
    
    # save refined label.csv
    df.to_csv("../../data/refinedFERPlus/"+mode+"/refined_label.csv", index=False, header=None)
    
    print("################")

Before drop miss images in [FER2013Train] mode, length of label.csv: 28558
After drop miss images, length of label.csv: 24997
             img_name            dims  0  1   2  3  4  5  6  7  Unknown  NF
0      fer0000000.png  (0, 0, 48, 48)  4  0   0  1  3  2  0  0        0   0
1      fer0000001.png  (0, 0, 48, 48)  6  0   1  1  0  0  0  0        2   0
2      fer0000002.png  (0, 0, 48, 48)  5  0   0  3  1  0  0  0        1   0
4      fer0000004.png  (0, 0, 48, 48)  9  0   0  1  0  0  0  0        0   0
5      fer0000005.png  (0, 0, 48, 48)  6  0   0  1  0  0  1  1        1   0
...               ...             ... .. ..  .. .. .. .. .. ..      ...  ..
28551  fer0028631.png  (0, 0, 48, 48)  5  0   0  2  3  0  0  0        0   0
28552  fer0028632.png  (0, 0, 48, 48)  0  0  10  0  0  0  0  0        0   0
28553  fer0028633.png  (0, 0, 48, 48)  1  0   2  0  0  0  7  0        0   0
28556  fer0028636.png  (0, 0, 48, 48)  5  0   0  1  0  1  0  0        3   0
28557  fer0028637.png  (0, 0, 48, 48) 

In [58]:
for mode in modes:
    miss_list = miss_dict[mode]
    df = pd.read_csv("../../data/refinedFERPlus/"+mode+"/label.csv", header=None)
    df.columns = columns
    
    print("Before drop miss images in [{}] mode, length of label.csv: {}".format(mode, len(df)))

Before drop miss images in [FER2013Train] mode, length of label.csv: 28558
Before drop miss images in [FER2013Test] mode, length of label.csv: 3573
Before drop miss images in [FER2013Valid] mode, length of label.csv: 3579


In [59]:
for mode in modes:
    miss_list = miss_dict[mode]
    df = pd.read_csv("../../data/refinedFERPlus/"+mode+"/refined_label.csv", header=None)
    df.columns = columns
    
    print("After drop miss images in [{}] mode, length of refined_label.csv: {}".format(mode, len(df)))

After drop miss images in [FER2013Train] mode, length of refined_label.csv: 24997
After drop miss images in [FER2013Test] mode, length of refined_label.csv: 3117
After drop miss images in [FER2013Valid] mode, length of refined_label.csv: 3103


In [60]:
df

,img_name,dims,0,1,2,3,4,5,6,7,Unknown,NF
0,fer0028638.png,"(0, 0, 48, 48)",4,0,0,0,1,0,0,4,1,0
1,fer0028639.png,"(0, 0, 48, 48)",1,0,0,1,0,2,0,6,0,0
2,fer0028641.png,"(0, 0, 48, 48)",5,5,0,0,0,0,0,0,0,0
3,fer0028642.png,"(0, 0, 48, 48)",0,10,0,0,0,0,0,0,0,0
4,fer0028643.png,"(0, 0, 48, 48)",0,10,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3098,fer0032215.png,"(0, 0, 48, 48)",3,0,1,4,0,0,0,0,2,0
3099,fer0032216.png,"(0, 0, 48, 48)",0,10,0,0,0,0,0,0,0,0
3100,fer0032217.png,"(0, 0, 48, 48)",4,0,0,4,1,0,0,0,1,0
3101,fer0032218.png,"(0, 0, 48, 48)",8,0,0,1,0,0,0,1,0,0
